#csv 합치기

In [6]:

import glob
import pandas as pd
from typing import List, Optional


def merge_csv_files(
    csv_dir: str,
    output_path: str = None,
    pattern: str = "*.csv",
    exclude_files: Optional[List[str]] = None
) -> pd.DataFrame:
    """
    지정된 디렉토리의 모든 CSV 파일을 하나로 합칩니다.

    Args:
        csv_dir: CSV 파일들이 있는 디렉토리 경로
        output_path: 출력 파일 경로 (None이면 저장 안 함)
        pattern: 파일 패턴 (기본: *.csv)
        exclude_files: 제외할 파일명 리스트

    Returns:
        합쳐진 DataFrame
    """
    if exclude_files is None:
        exclude_files = []

    # CSV 파일 목록 가져오기
    csv_files = glob.glob(os.path.join(csv_dir, pattern))
    csv_files = [f for f in csv_files if os.path.basename(f) not in exclude_files]
    csv_files.sort()

    if not csv_files:
        print(f"[WARNING] '{csv_dir}'에서 CSV 파일을 찾을 수 없습니다.")
        return pd.DataFrame()

    print(f"[INFO] {len(csv_files)}개의 CSV 파일 발견:")
    for f in csv_files:
        print(f"  - {os.path.basename(f)}")

    # 각 CSV 로드 및 통합
    dfs = []
    for csv_path in csv_files:
        try:
            df = pd.read_csv(csv_path)
            df['source_file'] = os.path.basename(csv_path)  # 출처 파일명 추가
            dfs.append(df)
            print(f"  ✓ {os.path.basename(csv_path)}: {len(df)} entries")
        except Exception as e:
            print(f"  ✗ {os.path.basename(csv_path)}: 로드 실패 - {e}")

    if not dfs:
        print("[ERROR] 로드된 CSV가 없습니다.")
        return pd.DataFrame()

    # 합치기
    merged_df = pd.concat(dfs, ignore_index=True)

    # 통계 출력
    print(f"\n[결과 요약]")
    print(f"  총 entries: {len(merged_df)}")
    if 'label' in merged_df.columns:
        pos_count = (merged_df['label'] == 1).sum()
        neg_count = (merged_df['label'] == 0).sum()
        print(f"  Positive (강조): {pos_count}")
        print(f"  Negative: {neg_count}")

    if 'video_path' in merged_df.columns:
        unique_videos = merged_df['video_path'].nunique()
        print(f"  고유 영상 수: {unique_videos}")

    # 저장
    if output_path:
        os.makedirs(os.path.dirname(output_path), exist_ok=True)

        # source_file 컬럼 제외하고 저장 (원본 포맷 유지)
        save_df = merged_df.drop(columns=['source_file'], errors='ignore')
        save_df.to_csv(output_path, index=False, encoding='utf-8')
        print(f"\n[저장 완료] {output_path}")

    return merged_df





In [7]:
# ===== 실행 =====
if __name__ == "__main__":
    # 설정 - 여기만 수정하세요
    CSV_DIR = "/home/stu/ai_project/영상csv"  # CSV 파일들이 있는 폴더
    OUTPUT_PATH = "/home/stu/ai_project/영상csv/최종csv.csv"  # 합친 결과 저장 경로

    # 합치기에서 제외할 파일 (이미 합쳐진 파일 등)
    EXCLUDE_FILES = ["최종csv.csv"]

    # 실행
    merged = merge_csv_files(
        csv_dir=CSV_DIR,
        output_path=OUTPUT_PATH,
        exclude_files=EXCLUDE_FILES
    )

    # 미리보기
    if not merged.empty:
        print(merged.head(10).to_string())

[INFO] 53개의 CSV 파일 발견:
  - 1.csv
  - 10.csv
  - 11.csv
  - 12.csv
  - 13.csv
  - 14.csv
  - 15.csv
  - 16.csv
  - 17.csv
  - 18.csv
  - 19.csv
  - 2.csv
  - 20.csv
  - 3.csv
  - 4.csv
  - 41.csv
  - 42.csv
  - 43.csv
  - 44.csv
  - 45.csv
  - 46.csv
  - 47.csv
  - 48.csv
  - 49.csv
  - 5.csv
  - 50.csv
  - 51.csv
  - 52.csv
  - 53.csv
  - 54.csv
  - 55.csv
  - 56.csv
  - 57.csv
  - 58.csv
  - 59.csv
  - 6.csv
  - 60.csv
  - 7.csv
  - 8.csv
  - 9.csv
  - 90.csv
  - 91.csv
  - 92.csv
  - 날씨1.csv
  - 빠더너스1.csv
  - 설민석1.csv
  - 세바시1.csv
  - 스티브잡스1_test.csv
  - 스티브잡스2.csv
  - 스티브잡스3.csv
  - 최태성1.csv
  - 최태성2_test.csv
  - 최태성3.csv
  ✓ 1.csv: 34 entries
  ✓ 10.csv: 25 entries
  ✓ 11.csv: 35 entries
  ✓ 12.csv: 33 entries
  ✓ 13.csv: 23 entries
  ✓ 14.csv: 28 entries
  ✓ 15.csv: 21 entries
  ✓ 16.csv: 23 entries
  ✓ 17.csv: 16 entries
  ✓ 18.csv: 20 entries
  ✓ 19.csv: 37 entries
  ✓ 2.csv: 29 entries
  ✓ 20.csv: 18 entries
  ✓ 3.csv: 18 entries
  ✓ 4.csv: 50 entries
  ✓ 41.csv: 56 entries
  ✓

### csv 경로 수정하기

In [10]:
import pandas as pd
import os

# 1. CSV 불러오기
input_path = "/home/stu/ai_project/영상csv/최종csv.csv"
df = pd.read_csv(input_path)

# 목표 경로 (서버의 실제 영상 위치)
NEW_DIR = "/home/stu/ai_project/영상raw"

# ---------------------------------------------------------
# 🧹 청소 및 경로 재설정 함수
# ---------------------------------------------------------
def clean_and_update_path(raw_path):
    # 1. 문자열로 변환
    path_str = str(raw_path)
    
    # 2. 윈도우 경로(\)를 리눅스 경로(/)로 통일 (그래야 자르기 쉬움)
    # 예: C:\Users\uni71\Desktop\영상raw\53.mp4 -> C:/Users/uni71/Desktop/영상raw/53.mp4
    path_str = path_str.replace('\\', '/')
    
    # 3. 경로 다 떼고 '파일명'만 추출
    # 예: .../53.mp4 -> 53.mp4 / .../43 -> 43
    filename = path_str.split('/')[-1]
    
    # 4. 확장자(.mp4) 없으면 붙여주기
    if not filename.lower().endswith('.mp4'):
        filename += '.mp4'
        
    # 5. 우리가 원하는 새 경로와 합치기
    # 결과: /home/stu/ai_project/영상raw/53.mp4
    return os.path.join(NEW_DIR, filename)

# ---------------------------------------------------------

# 2. 함수 적용
df["video_path"] = df["video_path"].apply(clean_and_update_path)

# 3. 결과 확인 (다양한 케이스가 잘 바뀌었는지 확인)
print("✅ 경로 수정 완료! (상위 5개 미리보기):")
for path in df["video_path"].head().tolist():
    print(path)

# 4. 저장하기
save_path = "/home/stu/ai_project/영상csv/최종csv.csv"
df.to_csv(save_path, index=False)
print(f"\n🎉 모든 경로 정리 끝! 저장된 파일: {save_path}")

✅ 경로 수정 완료! (상위 5개 미리보기):
/home/stu/ai_project/영상raw/1.mp4
/home/stu/ai_project/영상raw/1.mp4
/home/stu/ai_project/영상raw/1.mp4
/home/stu/ai_project/영상raw/1.mp4
/home/stu/ai_project/영상raw/1.mp4

🎉 모든 경로 정리 끝! 저장된 파일: /home/stu/ai_project/영상csv/최종csv.csv


In [ ]:
import os
import cv2
import csv
import random
import torch
import numpy as np
from collections import defaultdict
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# 기존 프로젝트 파일에서 가져올 것들
from config import GESTURE_CONFIG
from utils import load_gesture_model, VideoEmphasisDataset

# =============================================================================
# 1. 설정 (경로 및 모델 지정)
# =============================================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 평가할 CSV 파일
TEST_CSV_PATH = "/home/stu/ai_project/영상csv/100.csv"

# 평가할 모델 경로
MODEL_PATH = "/home/stu/ai_project/모델집합/gesture_model.pt"

# =============================================================================
# 2. 전처리 함수들 (제공해주신 코드 반영)
# =============================================================================

def load_label_entries(csv_path):
    entries = []
    if not os.path.exists(csv_path):
        print(f"[WARN] CSV file not found: {csv_path}")
        return []

    # 인코딩 문제 방지를 위한 처리 (utf-8 시도 후 cp949 시도)
    try:
        f = open(csv_path, newline='', encoding='utf-8')
        reader = csv.DictReader(f)
        next(reader) # 헤더 읽기 테스트
        f.seek(0)
    except UnicodeDecodeError:
        f.close()
        f = open(csv_path, newline='', encoding='cp949')

    with f:
        reader = csv.DictReader(f)
        for row in reader:
            entries.append({
                "video_id": row.get("video_id", "unknown"),
                "video_path": row["video_path"],
                "start_sec": float(row["start_sec"]),
                "end_sec": float(row["end_sec"]),
                "label": int(row["label"]),
            })
    print(f"[INFO] Loaded {len(entries)} positive entries from {csv_path}")
    return entries

def generate_negative_entries(positive_entries, num_neg_per_pos=1, clip_duration=2.0, min_gap=0.5):
    if not positive_entries:
        return []

    by_video = defaultdict(list)
    for e in positive_entries:
        by_video[e["video_path"]].append(e)

    all_negative = []

    for video_path, pos_list in by_video.items():
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"[WARNING] Cannot open video: {video_path}")
            continue

        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        video_duration = total_frames / fps if fps > 0 else 0
        cap.release()

        if video_duration < clip_duration:
            continue

        # 시간순 정렬
        sorted_pos = sorted(pos_list, key=lambda x: x['start_sec'])
        video_id = sorted_pos[0].get('video_id', 'unknown')
        negative_candidates = []

        # 1. 첫 번째 positive 이전 구간
        first_start = sorted_pos[0]['start_sec']
        if first_start > clip_duration + min_gap:
            t = 0.0
            while t + clip_duration <= first_start - min_gap:
                negative_candidates.append({
                    'video_id': video_id, 'video_path': video_path,
                    'start_sec': round(t, 3), 'end_sec': round(t + clip_duration, 3),
                    'label': 0
                })
                t += clip_duration

        # 2. Positive 구간 사이사이
        for i in range(len(sorted_pos) - 1):
            gap_start = sorted_pos[i]['end_sec'] + min_gap
            gap_end = sorted_pos[i + 1]['start_sec'] - min_gap

            if gap_end - gap_start >= clip_duration:
                t = gap_start
                while t + clip_duration <= gap_end:
                    negative_candidates.append({
                        'video_id': video_id, 'video_path': video_path,
                        'start_sec': round(t, 3), 'end_sec': round(t + clip_duration, 3),
                        'label': 0
                    })
                    t += clip_duration

        # 3. 마지막 positive 이후 구간
        last_end = sorted_pos[-1]['end_sec']
        if video_duration - last_end > clip_duration + min_gap:
            t = last_end + min_gap
            while t + clip_duration <= video_duration - min_gap:
                negative_candidates.append({
                    'video_id': video_id, 'video_path': video_path,
                    'start_sec': round(t, 3), 'end_sec': round(t + clip_duration, 3),
                    'label': 0
                })
                t += clip_duration

        # 샘플링 (Positive 개수 * 비율 만큼)
        target_num = int(len(pos_list) * num_neg_per_pos)
        if len(negative_candidates) > target_num:
            sampled = random.sample(negative_candidates, target_num)
        else:
            sampled = negative_candidates

        all_negative.extend(sampled)

    print(f"[INFO] Generated {len(all_negative)} negative entries (Ratio {num_neg_per_pos}:1)")
    return all_negative

def prepare_all_entries(csv_path, num_neg_per_pos=1, shuffle=True):
    # 1. 긍정 데이터 로드
    pos = load_label_entries(csv_path)
    
    # 2. 부정 데이터 자동 생성
    neg = generate_negative_entries(pos, num_neg_per_pos=num_neg_per_pos)
    
    # 3. 통합
    all_e = pos + neg
    if shuffle:
        random.shuffle(all_e)
        
    print(f"✅ 최종 데이터셋: 총 {len(all_e)}개 (Pos: {len(pos)}, Neg: {len(neg)})")
    return all_e, len(pos), len(neg)

# =============================================================================
# 3. 실행 및 평가 로직
# =============================================================================

if __name__ == "__main__":
    print(f"🖥️ Device: {device}")
    
    # 1. 데이터 준비 (제공해주신 함수 사용)
    # num_neg_per_pos=1 : 긍정 1개당 부정 1개 생성 (1:1 비율)
    all_entries, num_pos, num_neg = prepare_all_entries(TEST_CSV_PATH, num_neg_per_pos=1, shuffle=True)
    
    if len(all_entries) == 0:
        print("❌ 데이터가 없습니다. 경로를 확인하세요.")
        exit()

    # 2. 데이터셋 & 로더 생성
    test_dataset = VideoEmphasisDataset(
        all_entries,
        clip_len=GESTURE_CONFIG["clip_len"],
        resize_hw=GESTURE_CONFIG["resize_hw"]
    )
    test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=4)

    # 3. 모델 로드
    print(f"\n🧠 Loading Model: {MODEL_PATH}")
    model = load_gesture_model(MODEL_PATH, device)
    model.eval()

    # 4. 추론 실행
    print("🚀 Evaluating...")
    y_true = []
    y_pred = []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    # 5. 결과 리포트
    print("\n" + "="*50)
    print("🏆 [최종 성적표] 모델 성능 분석 결과")
    print("="*50)

    # 정확도
    acc = accuracy_score(y_true, y_pred)
    print(f"📊 정확도 (Accuracy): {acc * 100:.2f}%")

    # 상세 리포트
    target_names = ['비강조(Normal)', '강조(Emphasis)']
    print("\n📋 상세 리포트 (Precision, Recall, F1):")
    print(classification_report(y_true, y_pred, target_names=target_names, digits=4, zero_division=0))

    # 혼동 행렬
    cm = confusion_matrix(y_true, y_pred)
    # 2x2 행렬일 경우에만 상세 출력
    if cm.shape == (2, 2):
        tn, fp, fn, tp = cm.ravel()
        print("🔍 오답 노트 (Confusion Matrix):")
        print(f"--------------------------------------------------")
        print(f"               [예측: 비강조]   [예측: 강조]")
        print(f"[실제: 비강조]      {tn:<5} (정답)      {fp:<5} (설레발)")
        print(f"[실제: 강조]        {fn:<5} (놓침)      {tp:<5} (정답)")
        print(f"--------------------------------------------------")
        
        if fp > fn:
            print("👉 '과잉 탐지' 경향: 비강조 구간을 강조라고 잘못 잡는 경우가 더 많습니다.")
        elif fn > fp:
            print("👉 '소극적 탐지' 경향: 실제 강조 구간을 놓치는 경우가 더 많습니다.")
        else:
            print("👉 균형 잡힌 모델입니다.")
    else:
        print("🔍 Confusion Matrix:\n", cm)
    
    print("="*50)

🖥️ Device: cuda
[INFO] Loaded 82 positive entries from /home/stu/ai_project/영상csv/100.csv
[INFO] Generated 3 negative entries (Ratio 1:1)
✅ 최종 데이터셋: 총 85개 (Pos: 82, Neg: 3)

🧠 Loading Model: /home/stu/ai_project/모델집합/gesture_model.pt
Loaded gesture model from: /home/stu/ai_project/모델집합/gesture_model.pt
🚀 Evaluating...

🏆 [최종 성적표] 모델 성능 분석 결과
📊 정확도 (Accuracy): 83.53%

📋 상세 리포트 (Precision, Recall, F1):
              precision    recall  f1-score   support

 비강조(Normal)     0.0000    0.0000    0.0000         3
강조(Emphasis)     0.9595    0.8659    0.9103        82

    accuracy                         0.8353        85
   macro avg     0.4797    0.4329    0.4551        85
weighted avg     0.9256    0.8353    0.8781        85

🔍 오답 노트 (Confusion Matrix):
--------------------------------------------------
               [예측: 비강조]   [예측: 강조]
[실제: 비강조]      0     (정답)      3     (설레발)
[실제: 강조]        11    (놓침)      71    (정답)
--------------------------------------------------
👉 '소극적 탐지' 경향: 실제

#모델비교 r3d_18선정

In [15]:
# =============================================================================
# 🎬 3D CNN 모델 비교 실험 (Train/Val/Test 분리 + Safe Negative Sampling)
# r3d_18 vs mc3_18 vs r2plus1d_18 공정한 성능 비교
# =============================================================================

import os
import time
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import cv2
from collections import defaultdict
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from torchvision.models.video import (
    r3d_18, R3D_18_Weights,
    mc3_18, MC3_18_Weights,
    r2plus1d_18, R2Plus1D_18_Weights
)

# =============================================================================
# 📁 설정
# =============================================================================
# ⭐ 여기만 수정하세요!
TRAIN_CSV_PATH = "/home/stu/ai_project/영상csv/최종csv.csv"
TEST_CSV_PATH = "/home/stu/ai_project/영상csv/스티브잡스1_test.csv" # 없으면 None
BASE_DIR = os.getcwd()

# 하이퍼파라미터
CONFIG = {
    "clip_len": 16,
    "resize_hw": (112, 112),
    "batch_size": 4,
    "num_epochs": 10,
    "lr": 1e-4,
    "weight_decay": 1e-4,
    "num_neg_per_pos": 2, # 강조 1개당 비강조 2개 생성
    "seed": 42,
    "train_ratio": 0.6,
    "val_ratio": 0.2,
    "test_ratio": 0.2,
}

# 재현성
random.seed(CONFIG["seed"])
np.random.seed(CONFIG["seed"])
torch.manual_seed(CONFIG["seed"])
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(CONFIG["seed"])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️ Device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

# =============================================================================
# 📊 데이터 준비 함수들
# =============================================================================
def load_label_entries(csv_path):
    """CSV에서 라벨 로드"""
    import csv
    entries = []
    if not os.path.exists(csv_path):
        print(f"[ERROR] CSV not found: {csv_path}")
        return []
    with open(csv_path, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            entries.append({
                "video_id": row.get("video_id", os.path.basename(row["video_path"])),
                "video_path": row["video_path"],
                "start_sec": float(row["start_sec"]),
                "end_sec": float(row["end_sec"]),
                "label": int(row["label"]),
            })
    return entries

# ★ [통합됨] 안전하고 효율적인 비강조(0) 생성 함수
def generate_negative_entries(positive_entries, num_neg_per_pos=2, clip_duration=2.0, min_gap=0.1):
    """
    라벨된 구간(1)을 제외한 빈 공간(Free Gaps)에서 안전하게 비강조(0) 구간 생성
    """
    if not positive_entries:
        return []

    # 비디오별 그룹화
    by_video = defaultdict(list)
    for e in positive_entries:
        by_video[e["video_path"]].append(e)

    all_negative = []

    for video_path, pos_list in by_video.items():
        # 비디오 정보 읽기
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            # print(f"[WARNING] Cannot open video: {video_path}")
            continue
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        video_duration = total_frames / fps if fps > 0 else 0
        cap.release()

        if video_duration < clip_duration:
            continue

        # 사용 불가 구간 (Blocked) 계산: 강조 구간 + 앞뒤 마진(min_gap)
        blocked = []
        for e in pos_list:
            blocked.append((max(0, e["start_sec"] - min_gap), min(video_duration, e["end_sec"] + min_gap)))
        
        # 시간순 정렬
        blocked.sort(key=lambda x: x[0])

        # 빈 구간 (Free Gaps) 찾기
        free_gaps = []
        prev_end = 0.0
        for bs, be in blocked:
            if bs > prev_end + clip_duration: # 클립 하나 들어갈 틈이 있으면
                free_gaps.append((prev_end, bs))
            prev_end = max(prev_end, be)
        
        # 마지막 구간 체크
        if prev_end + clip_duration < video_duration:
            free_gaps.append((prev_end, video_duration))

        # 샘플링 (필요한 개수만큼)
        target_num = len(pos_list) * num_neg_per_pos
        
        # 모든 가능한 후보를 다 만든 뒤 랜덤 선택 (균등 분포 효과)
        candidates = []
        for gap_start, gap_end in free_gaps:
            t = gap_start
            while t + clip_duration <= gap_end:
                candidates.append((t, t + clip_duration))
                t += clip_duration 
        
        # 후보 중에서 랜덤 샘플링
        if len(candidates) > target_num:
            selected = random.sample(candidates, target_num)
        else:
            selected = candidates 
            
        # 결과 리스트에 추가
        video_id = pos_list[0].get('video_id', 'unknown')
        for s, e in selected:
            all_negative.append({
                'video_id': video_id,
                'video_path': video_path,
                'start_sec': round(s, 3),
                'end_sec': round(e, 3),
                'label': 0
            })

    # print(f"[INFO] Generated {len(all_negative)} negative entries")
    return all_negative


def split_by_video_3way(entries, train_ratio=0.6, val_ratio=0.2, seed=42):
    """비디오 ID 기준 데이터 분할"""
    random.seed(seed)
    by_video = defaultdict(list)
    for e in entries:
        vid = e.get('video_id', e.get('video_path', 'unknown'))
        by_video[vid].append(e)

    video_ids = list(by_video.keys())
    random.shuffle(video_ids)

    n_videos = len(video_ids)
    train_end = int(n_videos * train_ratio)
    val_end = int(n_videos * (train_ratio + val_ratio))

    train_vids = set(video_ids[:train_end])
    val_vids = set(video_ids[train_end:val_end])
    test_vids = set(video_ids[val_end:])

    train_entries, val_entries, test_entries = [], [], []

    for vid, ents in by_video.items():
        if vid in train_vids: train_entries.extend(ents)
        elif vid in val_vids: val_entries.extend(ents)
        else: test_entries.extend(ents)

    random.shuffle(train_entries)
    random.shuffle(val_entries)
    random.shuffle(test_entries)

    print(f"[INFO] Video Split: Train={len(train_vids)}, Val={len(val_vids)}, Test={len(test_vids)}")
    return train_entries, val_entries, test_entries

# =============================================================================
# 📦 Dataset & Model
# =============================================================================
class VideoEmphasisDataset(Dataset):
    def __init__(self, entries, clip_len=16, resize_hw=(112, 112)):
        self.entries = entries
        self.clip_len = clip_len
        self.resize_hw = resize_hw

    def __len__(self): return len(self.entries)

    def __getitem__(self, idx):
        entry = self.entries[idx]
        cap = cv2.VideoCapture(entry["video_path"])
        if not cap.isOpened():
            return torch.zeros(3, self.clip_len, *self.resize_hw), torch.tensor(entry["label"], dtype=torch.long)

        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        start_frame = int(entry["start_sec"] * fps)
        end_frame = int(entry["end_sec"] * fps)
        
        start_frame = max(0, min(start_frame, total_frames - 1))
        end_frame = max(0, min(end_frame, total_frames - 1))
        if end_frame <= start_frame: end_frame = min(start_frame + self.clip_len, total_frames - 1)
        
        seg_len = end_frame - start_frame + 1
        if seg_len >= self.clip_len:
            indices = np.linspace(start_frame, end_frame, num=self.clip_len, dtype=int)
        else:
            indices = np.zeros(self.clip_len, dtype=int)
            if seg_len > 0:
                base = np.linspace(start_frame, end_frame, num=seg_len, dtype=int)
                repeat = int(np.ceil(self.clip_len / seg_len))
                indices = np.tile(base, repeat)[:self.clip_len]

        frames = []
        for f in indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, f)
            ret, frame = cap.read()
            if not ret: frame = np.zeros((*self.resize_hw, 3), dtype=np.uint8)
            else: frame = cv2.resize(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), self.resize_hw)
            frames.append(frame)
        cap.release()
        
        x = torch.from_numpy(np.stack(frames)).float().permute(3, 0, 1, 2)
        x = (x / 255.0 - 0.5) / 0.5
        return x, torch.tensor(entry["label"], dtype=torch.long)

class GestureModel3D(nn.Module):
    def __init__(self, backbone_name='r3d_18', num_classes=2, pretrained=True):
        super().__init__()
        if backbone_name == 'r3d_18':
            weights = R3D_18_Weights.KINETICS400_V1 if pretrained else None
            self.backbone = r3d_18(weights=weights)
        elif backbone_name == 'mc3_18':
            weights = MC3_18_Weights.KINETICS400_V1 if pretrained else None
            self.backbone = mc3_18(weights=weights)
        elif backbone_name == 'r2plus1d_18':
            weights = R2Plus1D_18_Weights.KINETICS400_V1 if pretrained else None
            self.backbone = r2plus1d_18(weights=weights)
        
        self.backbone.fc = nn.Identity()
        # in_features 안전하게 가져오기
        dummy_input = torch.randn(1, 3, 16, 112, 112)
        with torch.no_grad():
            out = self.backbone(dummy_input)
        in_features = out.shape[1]
        
        self.classifier = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.classifier(self.backbone(x))
    
    def get_num_params(self): return sum(p.numel() for p in self.parameters())

def measure_inference_time(model, device, num_runs=10):
    model.eval()
    x = torch.randn(1, 3, 16, 112, 112).to(device)
    with torch.no_grad():
        for _ in range(5): model(x) # warmup
        times = []
        for _ in range(num_runs):
            s = time.time()
            model(x)
            if device.type == 'cuda': torch.cuda.synchronize()
            times.append(time.time() - s)
    return np.mean(times)*1000, np.std(times)*1000

# =============================================================================
# 🚀 실험 실행 (Training Loop)
# =============================================================================
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        correct += (out.argmax(1) == y).sum().item()
        total += y.size(0)
    return total_loss/len(loader), correct/total

def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    preds, targets = [], []
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            total_loss += criterion(out, y).item()
            preds.extend(out.argmax(1).cpu().numpy())
            targets.extend(y.cpu().numpy())
    
    acc = accuracy_score(targets, preds)
    f1 = f1_score(targets, preds, average='weighted', zero_division=0)
    return total_loss/len(loader), acc, f1, preds, targets

# 1. 데이터 로드 및 분할
print("\n[1/6] 데이터 준비...")
pos_entries = load_label_entries(TRAIN_CSV_PATH)
neg_entries = generate_negative_entries(pos_entries, CONFIG["num_neg_per_pos"]) # ★ 개선된 함수 사용
all_entries = pos_entries + neg_entries
random.shuffle(all_entries)

if TEST_CSV_PATH and os.path.exists(TEST_CSV_PATH):
    print(f"  ✅ 별도 Test CSV 사용: {TEST_CSV_PATH}")
    test_pos = load_label_entries(TEST_CSV_PATH)
    test_neg = generate_negative_entries(test_pos, CONFIG["num_neg_per_pos"]) # ★ 개선된 함수 사용
    test_entries = test_pos + test_neg
    train_entries, val_entries, _ = split_by_video_3way(all_entries, 0.8, 0.2, CONFIG["seed"])
else:
    train_entries, val_entries, test_entries = split_by_video_3way(all_entries, CONFIG["train_ratio"], CONFIG["val_ratio"], CONFIG["seed"])

def count_labels(entries):
    pos = sum(1 for e in entries if e['label'] == 1)
    neg = sum(1 for e in entries if e['label'] == 0)
    return pos, neg

train_pos, train_neg = count_labels(train_entries)
val_pos, val_neg = count_labels(val_entries)
test_pos, test_neg = count_labels(test_entries)

print(f"\n  📊 데이터 분포:")
print(f"     Train: {len(train_entries):4d} (Pos: {train_pos}, Neg: {train_neg})")
print(f"     Val:   {len(val_entries):4d} (Pos: {val_pos}, Neg: {val_neg})")
print(f"     Test:  {len(test_entries):4d} (Pos: {test_pos}, Neg: {test_neg})")

# 데이터로더
train_loader = DataLoader(VideoEmphasisDataset(train_entries, CONFIG["clip_len"], CONFIG["resize_hw"]), 
                          batch_size=CONFIG["batch_size"], shuffle=True, num_workers=2)
val_loader = DataLoader(VideoEmphasisDataset(val_entries, CONFIG["clip_len"], CONFIG["resize_hw"]), 
                        batch_size=CONFIG["batch_size"], shuffle=False, num_workers=2)
test_loader = DataLoader(VideoEmphasisDataset(test_entries, CONFIG["clip_len"], CONFIG["resize_hw"]), 
                         batch_size=CONFIG["batch_size"], shuffle=False, num_workers=2)

# 2. 모델 학습 및 비교
MODELS = ['r3d_18', 'mc3_18', 'r2plus1d_18']
results = {}
test_results = {} # 테스트 결과 저장용

# 클래스 가중치 계산 (Train 기준)
pos_weight = train_neg / train_pos if train_pos > 0 else 1.0
class_weights = torch.tensor([1.0, pos_weight]).to(device)

for name in MODELS:
    print(f"\n🔥 Training: {name}")
    model = GestureModel3D(name).to(device)
    crit = nn.CrossEntropyLoss(weight=class_weights)
    opt = optim.AdamW(model.parameters(), lr=CONFIG["lr"], weight_decay=CONFIG["weight_decay"])
    
    best_f1 = 0
    best_state = None
    
    history = {'train_loss': [], 'val_f1': []} # 히스토리 저장

    for ep in range(CONFIG["num_epochs"]):
        t_loss, t_acc = train_one_epoch(model, train_loader, opt, crit, device)
        v_loss, v_acc, v_f1, _, _ = evaluate(model, val_loader, crit, device)
        
        history['train_loss'].append(t_loss)
        history['val_f1'].append(v_f1)
        
        if v_f1 > best_f1:
            best_f1 = v_f1
            best_state = model.state_dict().copy()
            
        print(f"  Ep {ep+1}: Train {t_loss:.4f}/{t_acc:.2f} | Val {v_loss:.4f}/{v_acc:.2f} (F1 {v_f1:.2f})")
    
    # 학습 완료 정보 저장
    inf_ms, _ = measure_inference_time(model, device)
    results[name] = {
        'best_val_f1': best_f1, 
        'inf_time_ms': inf_ms, 
        'params': model.get_num_params(),
        'history': history,
        'train_time': 0 # (시간 측정 로직은 생략됨, 필요시 추가)
    }

    # Test 평가
    model.load_state_dict(best_state)
    _, test_acc, test_f1, preds, labels = evaluate(model, test_loader, crit, device)
    
    test_results[name] = {'test_acc': test_acc, 'test_f1': test_f1, 'test_preds': preds, 'test_labels': labels}
    print(f"  ✅ {name} Test Result -> Acc: {test_acc:.2f}, F1: {test_f1:.2f}")


# 3. 최종 결과 출력
print("\n" + "="*80)
print("🏆 최종 결과 요약")
print(f"{'Model':<15} {'Params(M)':<10} {'Time(ms)':<10} {'Val F1':<10} {'Test Acc':<10} {'Test F1':<10}")
print("-" * 80)
for name in MODELS:
    r = results[name]
    tr = test_results[name]
    print(f"{name:<15} {r['params']/1e6:<10.2f} {r['inf_time_ms']:<10.2f} {r['best_val_f1']:<10.4f} {tr['test_acc']:<10.4f} {tr['test_f1']:<10.4f}")
print("=" * 80)

# 최고 모델 선정
best_model = max(test_results.keys(), key=lambda x: test_results[x]['test_f1'])
print(f"\n🥇 최고 성능 모델 (Test 기준): {best_model}")

🖥️ Device: cuda
   GPU: Tesla P40

[1/6] 데이터 준비...
  ✅ 별도 Test CSV 사용: /home/stu/ai_project/영상csv/스티브잡스1_test.csv
[INFO] Video Split: Train=16, Val=4, Test=0

  📊 데이터 분포:
     Train: 5789 (Pos: 2631, Neg: 3158)
     Val:    784 (Pos: 299, Neg: 485)
     Test:    45 (Pos: 45, Neg: 0)

🔥 Training: r3d_18


[h264 @ 0x25e90a40] mmco: unref short failure
[h264 @ 0x25e90a40] mmco: unref short failure
[h264 @ 0x25e90a40] mmco: unref short failure
[h264 @ 0x25e90a40] mmco: unref short failure
[h264 @ 0x25e90a40] mmco: unref short failure
[h264 @ 0x25e90a40] mmco: unref short failure
[h264 @ 0x25e90a40] mmco: unref short failure
[h264 @ 0x25e90a40] mmco: unref short failure
[h264 @ 0x25e90a40] mmco: unref short failure
[h264 @ 0x25e90a40] mmco: unref short failure
[h264 @ 0x25e90a40] mmco: unref short failure
[h264 @ 0x25e90a40] mmco: unref short failure
[h264 @ 0x25e90a40] mmco: unref short failure
[h264 @ 0x25e90a40] mmco: unref short failure
[h264 @ 0x25e90a40] mmco: unref short failure
[h264 @ 0x25e90a40] mmco: unref short failure
[h264 @ 0x25e90a40] mmco: unref short failure
[h264 @ 0x25e90a40] mmco: unref short failure


KeyboardInterrupt: 

In [ ]:
import osnn
import json
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import librosa
from sklearn.preprocessing import StandardScaler
# =============================================================================
# 경로 설정
VIDEO_PATH = "/home/stu/ai_project/영상raw/test_jo.mp4"
MODEL_DIR = "/home/stu/ai_project/모델집합"

GESTURE_MODEL_PATH = os.path.join(MODEL_DIR, "gesture_model.pt")
AUDIO_MODEL_PATH = os.path.join(MODEL_DIR, "voice_model.pth")
TEXT_TENSOR_PATH = os.path.join(MODEL_DIR, "text_jo_torch_text.pt")
AUDIO_SCALER_PATH = os.path.join(MODEL_DIR, "audio_scaler.pkl")  # 있으면 사용
STT_JSON_PATH = "/home/stu/ai_project/결과/text.json"

# 출력 경로
OUTPUT_JSON = "/home/stu/ai_project/결과/test_jo_fusion_result.json"

# 가중치 설정 ( 조절 가능)
WEIGHTS = {
    "gesture": 0.35,
    "audio": 0.25,
    "text": 0.4,
}

# Gesture 설정
CLIP_LEN = 16
STRIDE = 8
RESIZE_HW = (112, 112)

# Audio 설정 (Mel-Spectrogram)
SR = 16000
N_FFT = 1024
HOP_LENGTH = 512
N_MELS = 80
WINDOW_SIZE = 50

# 디바이스
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️ Device: {device}")

# =============================================================================
# 🏗️ 모델 정의
# =============================================================================

# 1. Gesture Model (r3d_18 기반)
from torchvision.models.video import r3d_18

class GestureModel(nn.Module):
    def __init__(self, num_classes=2):
        super(GestureModel, self).__init__()
        self.backbone = r3d_18(weights=None)
        in_features = self.backbone.fc.in_features  # 512
        self.backbone.fc = nn.Identity()
        self.classifier = nn.Linear(in_features, num_classes)
    
    def forward(self, x, return_feature=False):
        features = self.backbone(x)  # (B, 512)
        logits = self.classifier(features)
        if return_feature:
            return logits, features
        return logits


# 2. Audio Model (BiLSTM, Mel-Spec 80차원, 4클래스)
class AudioBiLSTM(nn.Module):
    def __init__(self, input_size=80, hidden_size=64, num_layers=2, 
                 num_classes=4, dropout=0.2):
        super(AudioBiLSTM, self).__init__()
        
        self.lstm = nn.LSTM(
            input_size= input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=True
        )
        
        self.feature_dim = hidden_size * 2  # 128
        
        self.classifier = nn.Sequential(
            nn.Linear(self.feature_dim, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, num_classes)
        )
    
    def forward(self, x, return_feature=False):
        lstm_out, _ = self.lstm(x)
        features = lstm_out[:, -1, :]  # (B, 128)
        logits = self.classifier(features)
        if return_feature:
            return logits, features
        return logits


# 3. Text Model (Dummy - 768차원 입력)
class TextModel(nn.Module):
    def __init__(self, input_dim=768, num_classes=2):
        super(TextModel, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)
        self.feature_dim = input_dim
    
    def forward(self, x, return_feature=False):
        logits = self.fc(x)
        if return_feature:
            return logits, x
        return logits


# 4. Simple Fusion MLP
class SimpleFusionMLP(nn.Module):
    """
    간단한 Fusion MLP
    Input: gesture_feat(512) + audio_feat(128) + text_feat(768) = 1408
    Output: 강조 확률 (0~1)
    """
    def __init__(self, gesture_dim=512, audio_dim=128, text_dim=768, hidden_dim=256):
        super(SimpleFusionMLP, self).__init__()
        
        input_dim = gesture_dim + audio_dim + text_dim  # 1408
        
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim // 2, 1)  # 이진 분류
        )
    
    def forward(self, gesture_feat, audio_feat, text_feat):
        combined = torch.cat([gesture_feat, audio_feat, text_feat], dim=1)
        return self.net(combined)


# =============================================================================
# 🔧 유틸리티 함수
# =============================================================================

def extract_mel_spectrogram(y_audio, sr=16000, n_mels=80, n_fft=1024, hop_length=512):
    """Mel-Spectrogram 추출"""
    y_audio = librosa.util.normalize(y_audio)
    mel = librosa.feature.melspectrogram(
        y=y_audio, sr=sr, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length
    )
    mel_db = librosa.power_to_db(mel, ref=np.max)
    return mel_db.T  # (Time, n_mels)


def pad_or_truncate(features, window_size):
    """윈도우 크기에 맞게 패딩/자르기"""
    if len(features) < window_size:
        pad_len = window_size - len(features)
        features = np.pad(features, ((0, pad_len), (0, 0)), mode='constant')
    else:
        features = features[:window_size]
    return features


# =============================================================================
# 🚀 메인 실행
# =============================================================================
print("\n" + "="*70)
print("🎬 Simple MLP Fusion 추론")
print("="*70)

# 1. 모델 로드
print("\n[1/5] 📦 모델 로드...")

# Gesture Model
gesture_model = GestureModel(num_classes=2).to(device)
if os.path.exists(GESTURE_MODEL_PATH):
    state = torch.load(GESTURE_MODEL_PATH, map_location=device)
    # key 이름 호환성 처리
    new_state = {}
    for k, v in state.items():
        if 'fc' in k and 'backbone' not in k:
            new_state[k.replace('fc', 'classifier')] = v
        else:
            new_state[k] = v
    gesture_model.load_state_dict(new_state, strict=False)
    print(f"  ✅ Gesture Model 로드 완료")
else:
    print(f"  ⚠️ Gesture Model 없음: {GESTURE_MODEL_PATH}")
gesture_model.eval()

# Audio Model
audio_model = AudioBiLSTM(
    input_size=N_MELS, hidden_size=64, num_layers=2, num_classes=4, dropout=0.2
).to(device)
if os.path.exists(AUDIO_MODEL_PATH):
    audio_model.load_state_dict(torch.load(AUDIO_MODEL_PATH, map_location=device), strict=False)
    print(f"  ✅ Audio Model 로드 완료")
else:
    print(f"  ⚠️ Audio Model 없음: {AUDIO_MODEL_PATH}")
audio_model.eval()

# Text Tensors
text_tensors = []
if os.path.exists(TEXT_TENSOR_PATH):
    text_tensors = torch.load(TEXT_TENSOR_PATH, map_location=device, weights_only=False)
    if isinstance(text_tensors, list):
        print(f"  ✅ Text Tensors 로드 완료: {len(text_tensors)}개 문장")
    else:
        print(f"  ✅ Text Tensors 로드 완료: shape={text_tensors.shape}")
else:
    print(f"  ⚠️ Text Tensors 없음: {TEXT_TENSOR_PATH}")

# Text Model (Dummy)
text_model = TextModel(input_dim=768, num_classes=2).to(device)
text_model.eval()

# Fusion MLP (학습 안 된 상태 - 가중치 합산으로 대체)
fusion_model = SimpleFusionMLP().to(device)
fusion_model.eval()

# 2. 비디오 정보 로드
print("\n[2/5] 🎥 비디오 분석...")

cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise RuntimeError(f"비디오를 열 수 없습니다: {VIDEO_PATH}")

fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
duration = total_frames / fps

print(f"  영상: {os.path.basename(VIDEO_PATH)}")
print(f"  FPS: {fps:.2f}, 총 프레임: {total_frames}, 길이: {duration:.2f}초")

# 3. 오디오 로드
print("\n[3/5] 🎵 오디오 추출...")

try:
    y_audio, sr = librosa.load(VIDEO_PATH, sr=SR)
    print(f"  오디오 길이: {len(y_audio)/SR:.2f}초")
except Exception as e:
    print(f"  ⚠️ 오디오 로드 실패: {e}")
    y_audio = np.zeros(int(duration * SR))

# 4. 클립별 추론
print("\n[4/5] 🔍 클립별 추론 중...")

results = []
current_text_idx = 0
start_frame = 0
stt_cursor = 0

with torch.no_grad():
    while start_frame + CLIP_LEN <= total_frames:
        start_sec = start_frame / fps
        end_sec = (start_frame + CLIP_LEN) / fps
        
        # ----- Gesture -----
        frames = []
        for i in range(start_frame, start_frame + CLIP_LEN):
            cap.set(cv2.CAP_PROP_POS_FRAMES, i)
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, RESIZE_HW)
            frames.append(frame)
        
        if len(frames) < CLIP_LEN:
            break
        
        # 텐서 변환
        vid_np = np.array(frames, dtype=np.float32) / 255.0
        vid_np = np.transpose(vid_np, (3, 0, 1, 2))  # (C, T, H, W)
        vid_np = (vid_np - 0.5) / 0.5
        vid_tensor = torch.from_numpy(vid_np).unsqueeze(0).to(device)
        
        # Gesture 추론
        g_logits, g_feat = gesture_model(vid_tensor, return_feature=True)
        g_prob = F.softmax(g_logits, dim=1)[0, 1].item()  # 강조 확률
        
        # ----- Audio -----
        audio_start = int(start_sec * SR)
        audio_end = int(end_sec * SR)
        y_clip = y_audio[audio_start:audio_end]
        
        if len(y_clip) > 0:
            mel_feat = extract_mel_spectrogram(y_clip, SR, N_MELS, N_FFT, HOP_LENGTH)
            mel_feat = pad_or_truncate(mel_feat, WINDOW_SIZE)
            
            # 간단한 스케일링
            mel_feat = (mel_feat - mel_feat.mean()) / (mel_feat.std() + 1e-6)
            
            aud_tensor = torch.from_numpy(mel_feat).float().unsqueeze(0).to(device)
            a_logits, a_feat = audio_model(aud_tensor, return_feature=True)
            
            # Audio: 4클래스 중 비강조(0)가 아닌 확률 합
            a_probs = F.softmax(a_logits, dim=1)[0]
            a_class = torch.argmax(a_probs).item()
            a_prob = 1.0 - a_probs[0].item()  # 강조 확률 (1 - 비강조)
        else:
            a_feat = torch.zeros(1, 128).to(device)
            a_prob = 0.0
            a_class = 0
        
        # ----- Text -----
        t_feat = torch.zeros(1, 768).to(device)
        t_prob = 0.0
        matched_text_idx = -1
        if len(text_tensors) > 0 and len(stt_segments) > 0:
            # 현재 클립 시간(start_sec)이 포함되는 STT 세그먼트 찾기
            # (stt_segments는 시간순 정렬되어 있다고 가정)
            for i in range(stt_cursor, len(stt_segments)):
                seg = stt_segments[i]
                # 세그먼트: { "start": 0.0, "end": 2.5, "text": "..." }
                seg_start = seg.get('start', 0)
                seg_end = seg.get('end', 0)
                
                # 클립의 중간 지점이 자막 구간 안에 있는지 확인
                clip_mid = (start_sec + end_sec) / 2
                
                if seg_start <= clip_mid <= seg_end:
                    matched_text_idx = i
                    stt_cursor = i  # 다음 검색은 여기서부터
                    break
                
                if seg_start > end_sec: # 이미 지나감
                    break
            
            # 매칭된 텍스트가 있으면 해당 텐서 사용
            if matched_text_idx != -1 and matched_text_idx < len(text_tensors):
                t_tensor = text_tensors[matched_text_idx]
                
                # 차원 맞추기
                if isinstance(t_tensor, torch.Tensor):
                    if t_tensor.dim() == 3: t_feat = t_tensor.mean(dim=1).to(device)
                    elif t_tensor.dim() == 2: t_feat = t_tensor.mean(dim=0, keepdim=True).to(device)
                    else: t_feat = t_tensor.unsqueeze(0).to(device)
                
                # 임시: 텍스트 점수 (모델이 있다면 모델 출력, 없으면 휴리스틱)
                # t_logits, _ = text_model(t_feat, return_feature=True)
                # t_prob = F.softmax(t_logits, dim=1)[0, 1].item()
                
                # 텐서의 크기(norm)를 '중요도'로 가정하는 임시 로직 (실제 모델 사용 시 교체)
                t_prob = min(1.0, t_feat.norm().item() / 50.0)
        
        # ----- Fusion (가중치 합산) -----
        fusion_score = (
            WEIGHTS["gesture"] * g_prob +
            WEIGHTS["audio"] * a_prob +
            WEIGHTS["text"] * t_prob
        )
        
        # 결과 저장
        results.append({
            "start_sec": round(start_sec, 2),
            "end_sec": round(end_sec, 2),
            "gesture_prob": round(g_prob, 4),
            "audio_prob": round(a_prob, 4),
            "audio_class": a_class,
            "text_prob": round(t_prob, 4),
            "fusion_score": round(fusion_score, 4),
            "is_emphasis": fusion_score >= 0.5
        })
        
        start_frame += STRIDE

cap.release()
print(f"  ✅ {len(results)}개 클립 분석 완료")

# 5. 결과 저장 및 통계
print("\n[5/5] 📊 결과 분석...")

# 통계 계산
gesture_scores = [r["gesture_prob"] for r in results]
audio_scores = [r["audio_prob"] for r in results]
text_scores = [r["text_prob"] for r in results]
fusion_scores = [r["fusion_score"] for r in results]
emphasis_count = sum(1 for r in results if r["is_emphasis"])

print(f"\n{'='*60}")
print("📊 전체 통계")
print(f"{'='*60}")
print(f"  총 클립 수: {len(results)}")
print(f"  강조 클립 수: {emphasis_count} ({emphasis_count/len(results)*100:.1f}%)")
print(f"\n  [Gesture] 평균: {np.mean(gesture_scores):.4f}, 최대: {np.max(gesture_scores):.4f}")
print(f"  [Audio]   평균: {np.mean(audio_scores):.4f}, 최대: {np.max(audio_scores):.4f}")
print(f"  [Text]    평균: {np.mean(text_scores):.4f}, 최대: {np.max(text_scores):.4f}")
print(f"  [Fusion]  평균: {np.mean(fusion_scores):.4f}, 최대: {np.max(fusion_scores):.4f}")

# 가중치 정보
print(f"\n  가중치: Gesture={WEIGHTS['gesture']}, Audio={WEIGHTS['audio']}, Text={WEIGHTS['text']}")

# 총점 계산 (전체 평균)
total_score = np.mean(fusion_scores) * 100
print(f"\n{'='*60}")
print(f"🏆 영상 총점: {total_score:.2f} / 100")
print(f"{'='*60}")

# 강조 구간 요약
print(f"\n📋 강조 구간 (fusion >= 0.5):")
emphasis_segments = []
in_emphasis = False
seg_start = 0

for r in results:
    if r["is_emphasis"] and not in_emphasis:
        seg_start = r["start_sec"]
        in_emphasis = True
    elif not r["is_emphasis"] and in_emphasis:
        emphasis_segments.append((seg_start, r["start_sec"]))
        in_emphasis = False

if in_emphasis:
    emphasis_segments.append((seg_start, results[-1]["end_sec"]))

if emphasis_segments:
    for i, (s, e) in enumerate(emphasis_segments[:10]):  # 최대 10개
        print(f"  {i+1}. {s:.2f}s ~ {e:.2f}s (길이: {e-s:.2f}s)")
    if len(emphasis_segments) > 10:
        print(f"  ... 외 {len(emphasis_segments) - 10}개")
else:
    print("  (강조 구간 없음)")

# JSON 저장
os.makedirs(os.path.dirname(OUTPUT_JSON), exist_ok=True)

output_data = {
    "video_path": VIDEO_PATH,
    "duration_sec": round(duration, 2),
    "total_clips": len(results),
    "emphasis_clips": emphasis_count,
    "weights": WEIGHTS,
    "statistics": {
        "gesture_mean": round(np.mean(gesture_scores), 4),
        "audio_mean": round(np.mean(audio_scores), 4),
        "text_mean": round(np.mean(text_scores), 4),
        "fusion_mean": round(np.mean(fusion_scores), 4),
    },
    "total_score": round(total_score, 2),
    "emphasis_segments": [{"start_sec": s, "end_sec": e} for s, e in emphasis_segments],
    "clips": results
}

with open(OUTPUT_JSON, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

print(f"\n✅ 결과 저장: {OUTPUT_JSON}")

print("\n" + "="*70)
print("🎬 Simple MLP Fusion 추론 (STT 연동 버전)")
print("="*70)

# -------------------------------------------------------------
# ★ [핵심 추가] STT JSON 로드
# -------------------------------------------------------------
stt_segments = []
if os.path.exists(STT_JSON_PATH):
    with open(STT_JSON_PATH, 'r', encoding='utf-8') as f:
        stt_data = json.load(f)
        # 구조에 따라 다를 수 있음 (예: data['segments'] 또는 바로 리스트)
        if isinstance(stt_data, dict) and 'segments' in stt_data:
            stt_segments = stt_data['segments']
        elif isinstance(stt_data, list):
            stt_segments = stt_data
        
    print(f"  ✅ STT 데이터 로드 완료: {len(stt_segments)}개 문장")
else:
    print(f"  ⚠️ STT 파일 없음: {STT_JSON_PATH} (텍스트 매칭 불가능)")

# -------------------------------------------------------------
# =============================================================================
# 📋 결과 샘플 출력
# =============================================================================
print(f"\n{'='*60}")
print("📋 클립별 결과 샘플 (처음 10개)")
print(f"{'='*60}")
print(f"{'Start':>7} {'End':>7} {'Gesture':>8} {'Audio':>8} {'Text':>8} {'Fusion':>8} {'강조':>4}")
print("-" * 60)

for r in results[:10]:
    emph = "✓" if r["is_emphasis"] else ""
    print(f"{r['start_sec']:>7.2f} {r['end_sec']:>7.2f} "
          f"{r['gesture_prob']:>8.4f} {r['audio_prob']:>8.4f} "
          f"{r['text_prob']:>8.4f} {r['fusion_score']:>8.4f} {emph:>4}")

print(f"\n🎉 추론 완료!")

🖥️ Device: cuda

🎬 Simple MLP Fusion 추론

[1/5] 📦 모델 로드...
  ✅ Gesture Model 로드 완료
  ✅ Audio Model 로드 완료
  ✅ Text Tensors 로드 완료: 46개 문장

[2/5] 🎥 비디오 분석...
  영상: test_jo.mp4
  FPS: 29.97, 총 프레임: 4268, 길이: 142.41초

[3/5] 🎵 오디오 추출...


/tmp/ipykernel_2537931/3663372167.py:242: UserWarning: PySoundFile failed. Trying audioread instead.
  y_audio, sr = librosa.load(VIDEO_PATH, sr=SR)
/home/stu/anaconda3/envs/ai_project_env/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


  오디오 길이: 142.52초

[4/5] 🔍 클립별 추론 중...
  ✅ 532개 클립 분석 완료

[5/5] 📊 결과 분석...

📊 전체 통계
  총 클립 수: 532
  강조 클립 수: 83 (15.6%)

  [Gesture] 평균: 0.1680, 최대: 0.9898
  [Audio]   평균: 0.7951, 최대: 0.7962
  [Text]    평균: 0.3557, 최대: 0.3801
  [Fusion]  평균: 0.3999, 최대: 0.6919

  가중치: Gesture=0.35, Audio=0.25, Text=0.4

🏆 영상 총점: 39.99 / 100

📋 강조 구간 (fusion >= 0.5):
  1. 1.07s ~ 1.33s (길이: 0.26s)
  2. 2.14s ~ 2.40s (길이: 0.26s)
  3. 12.28s ~ 14.15s (길이: 1.87s)
  4. 16.28s ~ 16.55s (길이: 0.27s)
  5. 26.43s ~ 26.69s (길이: 0.26s)
  6. 34.43s ~ 35.50s (길이: 1.07s)
  7. 36.84s ~ 37.10s (길이: 0.26s)
  8. 39.77s ~ 40.31s (길이: 0.54s)
  9. 40.57s ~ 41.11s (길이: 0.54s)
  10. 47.78s ~ 48.31s (길이: 0.53s)
  ... 외 25개

✅ 결과 저장: /home/stu/ai_project/결과/test_jo_fusion_result.json

📋 클립별 결과 샘플 (처음 10개)
  Start     End  Gesture    Audio     Text   Fusion   강조
------------------------------------------------------------
   0.00    0.53   0.0135   0.7958   0.3330   0.3368     
   0.27    0.80   0.0319   0.7944   0.3330   0.3429

In [ ]:
# # =============================================================================
# # 🎬 멀티모달 발표 분석 + LLM 피드백 + 시각화 영상 생성
# # =============================================================================
# #
# # 파이프라인:
# # 1. STT JSON 로드 (문장 + 시간 정보)
# # 2. 멀티모달 분석 (Gesture + Audio + Text)
# # 3. 강조 구간 추출 + LLM 피드백 생성
# # 4. 시각화 영상 생성
# #
# # =============================================================================

# import os
# import json
# import numpy as np
# import cv2
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import librosa
# from datetime import datetime

# # =============================================================================
# # ⚙️ 설정
# # =============================================================================
# # 경로 설정
# VIDEO_PATH = "/home/stu/ai_project/영상raw/test_jo.mp4"
# MODEL_DIR = "/home/stu/ai_project/모델집합"
# STT_JSON_PATH = "/home/stu/ai_project/결과/text.json"

# GESTURE_MODEL_PATH = os.path.join(MODEL_DIR, "gesture_model.pt")
# AUDIO_MODEL_PATH = os.path.join(MODEL_DIR, "voice_model.pth")
# TEXT_TENSOR_PATH = os.path.join(MODEL_DIR, "text_jo_torch_text.pt")

# # 출력 경로
# OUTPUT_DIR = "/home/stu/ai_project/결과"
# OUTPUT_JSON = os.path.join(OUTPUT_DIR, "test_jo_analysis_result.json")
# OUTPUT_VIDEO = os.path.join(OUTPUT_DIR, "test_jo_visualization.mp4")
# OUTPUT_FEEDBACK = os.path.join(OUTPUT_DIR, "test_jo_llm_feedback.json")

# # 가중치 설정 (⭐ 조절 가능!)
# WEIGHTS = {
#     "gesture": 0.4,
#     "audio": 0.35,
#     "text": 0.25,
# }

# # 강조 임계값
# EMPHASIS_THRESHOLD = 0.3

# # Gesture 설정
# CLIP_LEN = 16
# STRIDE = 8
# RESIZE_HW = (112, 112)

# # Audio 설정 (Mel-Spectrogram)
# SR = 16000
# N_FFT = 1024
# HOP_LENGTH = 512
# N_MELS = 80
# WINDOW_SIZE = 50

# # LLM API 설정 (⭐ 본인 API 키로 변경!)
# # OpenAI 또는 Claude API 사용
# USE_OPENAI = True  # False면 Claude 사용
# OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", "your-api-key-here")
# ANTHROPIC_API_KEY = os.environ.get("ANTHROPIC_API_KEY", "your-api-key-here")

# # 디바이스
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"🖥️ Device: {device}")

# # 출력 디렉토리 생성
# os.makedirs(OUTPUT_DIR, exist_ok=True)


# # =============================================================================
# # 🏗️ 모델 정의
# # =============================================================================
# from torchvision.models.video import r3d_18

# class GestureModel(nn.Module):
#     def __init__(self, num_classes=2):
#         super(GestureModel, self).__init__()
#         self.backbone = r3d_18(weights=None)
#         in_features = self.backbone.fc.in_features
#         self.backbone.fc = nn.Identity()
#         self.classifier = nn.Linear(in_features, num_classes)
    
#     def forward(self, x, return_feature=False):
#         features = self.backbone(x)
#         logits = self.classifier(features)
#         if return_feature:
#             return logits, features
#         return logits


# class AudioBiLSTM(nn.Module):
#     def __init__(self, input_dim=80, hidden_dim=64, num_layers=2, 
#                  num_classes=4, dropout=0.2):
#         super(AudioBiLSTM, self).__init__()
#         self.lstm = nn.LSTM(
#             input_size=input_dim, hidden_size=hidden_dim,
#             num_layers=num_layers, batch_first=True,
#             dropout=dropout if num_layers > 1 else 0, bidirectional=True
#         )
#         self.feature_dim = hidden_dim * 2
#         self.classifier = nn.Sequential(
#             nn.Linear(self.feature_dim, 64), nn.ReLU(),
#             nn.Dropout(dropout), nn.Linear(64, num_classes)
#         )
    
#     def forward(self, x, return_feature=False):
#         lstm_out, _ = self.lstm(x)
#         features = lstm_out[:, -1, :]
#         logits = self.classifier(features)
#         if return_feature:
#             return logits, features
#         return logits


# # =============================================================================
# # 🔧 유틸리티 함수
# # =============================================================================
# def extract_mel_spectrogram(y_audio, sr=16000, n_mels=80, n_fft=1024, hop_length=512):
#     """Mel-Spectrogram 추출"""
#     y_audio = librosa.util.normalize(y_audio)
#     mel = librosa.feature.melspectrogram(
#         y=y_audio, sr=sr, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length
#     )
#     mel_db = librosa.power_to_db(mel, ref=np.max)
#     return mel_db.T


# def pad_or_truncate(features, window_size):
#     """윈도우 크기에 맞게 패딩/자르기"""
#     if len(features) < window_size:
#         pad_len = window_size - len(features)
#         features = np.pad(features, ((0, pad_len), (0, 0)), mode='constant')
#     else:
#         features = features[:window_size]
#     return features


# def load_stt_json(path):
#     """STT JSON 로드 (딕셔너리 안에 리스트 들어있는 구조까지 자동 처리)"""
#     if not os.path.exists(path):
#         print(f"⚠️ STT 파일 없음: {path}")
#         return []
    
#     with open(path, 'r', encoding='utf-8') as f:
#         data = json.load(f)

#     # 1) 최상단이 바로 리스트인 경우
#     if isinstance(data, list):
#         print(f"✅ STT: top-level list, count={len(data)}")
#         return data

#     # 2) 최상단이 dict인 경우 → 안에 들어있는 list를 찾아서 사용
#     if isinstance(data, dict):
#         list_keys = [k for k, v in data.items() if isinstance(v, list)]

#         if not list_keys:
#             print(f"⚠️ STT JSON 안에 리스트 타입이 없음. keys={list(data.keys())}")
#             return []

#         # 리스트가 여러 개면 첫 번째를 사용 (필요하면 여기서 원하는 키 골라서 쓰면 됨)
#         key = list_keys[0]
#         segments = data[key]
#         print(f"✅ STT: data['{key}'] 사용, count={len(segments)}")
#         return segments

#     print(f"⚠️ STT JSON 형식 인식 불가: {type(data)}")
#     return []


# def find_key_word(text):
#     """문장에서 키워드 추출 (간단한 휴리스틱)"""
#     # 명사/주요 단어 추출 (간단 버전)
#     # 실제로는 형태소 분석기 사용 권장
#     words = text.replace(".", "").replace(",", "").replace("?", "").split()
    
#     # 2글자 이상, 조사 제외
#     stopwords = ["이", "가", "을", "를", "은", "는", "에", "에서", "으로", "로", "와", "과", 
#                  "의", "도", "만", "까지", "부터", "이런", "그런", "저런", "하는", "있는"]
    
#     keywords = [w for w in words if len(w) >= 2 and w not in stopwords]
    
#     # 마지막 주요 단어 반환 (문장 끝에 핵심이 오는 경우가 많음)
#     if keywords:
#         return keywords[-1]
#     return words[-1] if words else ""


# # =============================================================================
# # 🤖 LLM 피드백 생성
# # =============================================================================
# def generate_llm_feedback_openai(emphasis_segments, video_duration):
#     """OpenAI API로 피드백 생성"""
#     try:
#         import openai
#         openai.api_key = OPENAI_API_KEY
        
#         # 프롬프트 구성
#         segments_text = "\n".join([
#             f"- [{s['start_sec']:.1f}s ~ {s['end_sec']:.1f}s] "
#             f"(강조점수: {s['emphasis_score']:.2f}) \"{s['text']}\""
#             for s in emphasis_segments
#         ])
        
#         prompt = f"""당신은 프레젠테이션 코칭 전문가입니다.

# 다음은 발표 영상에서 강조가 필요한 구간들입니다:
# 영상 길이: {video_duration:.1f}초

# 강조 구간:
# {segments_text}

# 각 구간에 대해 다음 형식으로 분석해주세요:
# 1. 왜 이 부분이 강조되어야 하는지 (reason)
# 2. 어떤 키워드가 핵심인지 (key_word)
# 3. 개선을 위한 조언

# JSON 배열 형식으로 응답해주세요:
# [
#   {{"index": 1, "reason": "이유", "key_word": "키워드", "advice": "조언"}},
#   ...
# ]
# """
        
#         response = openai.ChatCompletion.create(
#             model="gpt-4",
#             messages=[{"role": "user", "content": prompt}],
#             temperature=0.7,
#             max_tokens=2000
#         )
        
#         result = response.choices[0].message.content
        
#         # JSON 파싱 시도
#         try:
#             # JSON 부분만 추출
#             import re
#             json_match = re.search(r'\[.*\]', result, re.DOTALL)
#             if json_match:
#                 return json.loads(json_match.group())
#         except:
#             pass
        
#         return result
        
#     except Exception as e:
#         print(f"⚠️ OpenAI API 에러: {e}")
#         return None


# def generate_llm_feedback_anthropic(emphasis_segments, video_duration):
#     """Anthropic Claude API로 피드백 생성"""
#     try:
#         import anthropic
#         client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)
        
#         segments_text = "\n".join([
#             f"- [{s['start_sec']:.1f}s ~ {s['end_sec']:.1f}s] "
#             f"(강조점수: {s['emphasis_score']:.2f}) \"{s['text']}\""
#             for s in emphasis_segments
#         ])
        
#         prompt = f"""당신은 프레젠테이션 코칭 전문가입니다.

# 다음은 발표 영상에서 강조가 필요한 구간들입니다:
# 영상 길이: {video_duration:.1f}초

# 강조 구간:
# {segments_text}

# 각 구간에 대해 다음 JSON 형식으로 분석해주세요:
# [
#   {{"index": 1, "reason": "왜 강조해야 하는지", "key_word": "핵심 키워드", "advice": "개선 조언"}},
#   ...
# ]

# JSON만 출력하세요."""

#         response = client.messages.create(
#             model="claude-sonnet-4-20250514",
#             max_tokens=2000,
#             messages=[{"role": "user", "content": prompt}]
#         )
        
#         result = response.content[0].text
        
#         try:
#             import re
#             json_match = re.search(r'\[.*\]', result, re.DOTALL)
#             if json_match:
#                 return json.loads(json_match.group())
#         except:
#             pass
        
#         return result
        
#     except Exception as e:
#         print(f"⚠️ Anthropic API 에러: {e}")
#         return None


# def generate_feedback_local(emphasis_segments):
#     """LLM 없이 로컬에서 피드백 생성 (규칙 기반)"""
#     feedback = []
    
#     reason_templates = [
#         "주제 도입 문장으로 청중의 공감 유도",
#         "핵심 개념 설명으로 중요도 높음",
#         "결론 또는 요약 부분으로 강조 필요",
#         "질문 형식으로 청중 참여 유도",
#         "구체적 예시로 이해도 향상",
#         "전환 구간으로 흐름 강조",
#         "핵심 주장 제시",
#         "감정적 호소로 청중 몰입 유도",
#     ]
    
#     for i, seg in enumerate(emphasis_segments):
#         text = seg.get("text", "")
#         score = seg.get("emphasis_score", 0)
        
#         # 키워드 추출
#         key_word = find_key_word(text)
        
#         # 이유 선택 (문장 특성에 따라)
#         if "?" in text:
#             reason = "질문 형식으로 청중 참여 유도 및 문제 제기"
#         elif any(kw in text for kw in ["그러니까", "즉", "결국"]):
#             reason = "요약/정리 구간으로 핵심 내용 강조"
#         elif any(kw in text for kw in ["예를 들어", "예시", "사례"]):
#             reason = "구체적 예시로 이해도 향상"
#         elif score > 0.7:
#             reason = "높은 강조 점수로 핵심 메시지 전달"
#         else:
#             reason = reason_templates[i % len(reason_templates)]
        
#         feedback.append({
#             "index": i + 1,
#             "start_sec": seg["start_sec"],
#             "end_sec": seg["end_sec"],
#             "text": text,
#             "emphasis_score": round(score, 2),
#             "key_word": key_word,
#             "reason": reason
#         })
    
#     return feedback


# # =============================================================================
# # 🎥 시각화 영상 생성
# # =============================================================================
# def create_visualization_video(video_path, emphasis_data, output_path):
#     """강조 구간 시각화 영상 생성"""
#     cap = cv2.VideoCapture(video_path)
#     if not cap.isOpened():
#         print(f"⚠️ 비디오를 열 수 없습니다: {video_path}")
#         return False
    
#     fps = cap.get(cv2.CAP_PROP_FPS)
#     width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#     height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
#     total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
#     # VideoWriter
#     fourcc = cv2.VideoWriter_fourcc(*'mp4v')
#     out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
#     # 강조 구간을 빠르게 검색하기 위한 준비
#     def get_emphasis_info(current_sec):
#         for seg in emphasis_data:
#             if seg["start_sec"] <= current_sec <= seg["end_sec"]:
#                 return seg
#         return None
    
#     print(f"  🎥 영상 생성 중... ({total_frames} frames)")
    
#     frame_idx = 0
#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             break
        
#         current_sec = frame_idx / fps
#         seg_info = get_emphasis_info(current_sec)
        
#         # 오버레이
#         overlay = frame.copy()
        
#         if seg_info:
#             # 강조 구간: 빨간 테두리 + 상단 바
#             color = (0, 0, 255)  # BGR - 빨강
#             score = seg_info.get("emphasis_score", 0)
#             text = seg_info.get("text", "")[:50]  # 최대 50자
#             key_word = seg_info.get("key_word", "")
            
#             # 상단 바
#             cv2.rectangle(overlay, (0, 0), (width, 80), color, -1)
            
#             # 텍스트
#             cv2.putText(overlay, f"EMPHASIS ({score:.2f})", (10, 30),
#                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
            
#             if key_word:
#                 cv2.putText(overlay, f"Key: {key_word}", (10, 60),
#                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
            
#             # 테두리
#             cv2.rectangle(overlay, (5, 5), (width-5, height-5), color, 4)
            
#             # 하단 자막 바
#             cv2.rectangle(overlay, (0, height-60), (width, height), (0, 0, 0), -1)
#             # 한글 표시를 위해 영어로 대체 (OpenCV 한글 제한)
#             cv2.putText(overlay, text[:40], (10, height-25),
#                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
#         else:
#             # 일반 구간: 녹색 상단 바
#             color = (0, 128, 0)  # BGR - 녹색
#             cv2.rectangle(overlay, (0, 0), (width, 40), color, -1)
#             cv2.putText(overlay, "NORMAL", (10, 28),
#                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
#         # 타임코드
#         time_str = f"{int(current_sec//60):02d}:{int(current_sec%60):02d}"
#         cv2.putText(overlay, time_str, (width-80, 28),
#                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
#         # 블렌딩
#         alpha = 0.7
#         frame = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)
        
#         out.write(frame)
#         frame_idx += 1
        
#         # 진행률 표시
#         if frame_idx % 500 == 0:
#             progress = frame_idx / total_frames * 100
#             print(f"    진행률: {progress:.1f}%")
    
#     cap.release()
#     out.release()
    
#     print(f"  ✅ 영상 저장 완료: {output_path}")
#     return True


# # =============================================================================
# # 🚀 메인 실행
# # =============================================================================
# def main():
#     print("\n" + "="*70)
#     print("🎬 멀티모달 발표 분석 시스템")
#     print("="*70)
    
#     # 1. 모델 로드
#     print("\n[1/6] 📦 모델 로드...")
    
#     # Gesture Model
#     gesture_model = GestureModel(num_classes=2).to(device)
#     if os.path.exists(GESTURE_MODEL_PATH):
#         state = torch.load(GESTURE_MODEL_PATH, map_location=device)
#         new_state = {}
#         for k, v in state.items():
#             if 'fc' in k and 'backbone' not in k:
#                 new_state[k.replace('fc', 'classifier')] = v
#             else:
#                 new_state[k] = v
#         gesture_model.load_state_dict(new_state, strict=False)
#         print(f"  ✅ Gesture Model 로드")
#     gesture_model.eval()
    
#     # Audio Model
#     audio_model = AudioBiLSTM(
#         input_dim=N_MELS, hidden_dim=64, num_layers=2, num_classes=4
#     ).to(device)
#     if os.path.exists(AUDIO_MODEL_PATH):
#         audio_model.load_state_dict(
#             torch.load(AUDIO_MODEL_PATH, map_location=device), strict=False
#         )
#         print(f"  ✅ Audio Model 로드")
#     audio_model.eval()
    
#     # Text Tensors
#     text_tensors = []
#     if os.path.exists(TEXT_TENSOR_PATH):
#         text_tensors = torch.load(TEXT_TENSOR_PATH, map_location=device, weights_only=False)
#         print(f"  ✅ Text Tensors 로드: {len(text_tensors)}개")
    
#     # 2. STT 로드
#     print("\n[2/6] 📝 STT 데이터 로드...")
#     stt_segments = load_stt_json(STT_JSON_PATH)
#     print(f"  ✅ {len(stt_segments)}개 문장 로드")
    
#     # 3. 비디오 정보
#     print("\n[3/6] 🎥 비디오 분석...")
#     cap = cv2.VideoCapture(VIDEO_PATH)
#     fps = cap.get(cv2.CAP_PROP_FPS)
#     total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#     duration = total_frames / fps
#     print(f"  영상: {os.path.basename(VIDEO_PATH)}")
#     print(f"  길이: {duration:.2f}초, FPS: {fps:.2f}")
    
#     # 오디오 로드
#     try:
#         y_audio, _ = librosa.load(VIDEO_PATH, sr=SR)
#         print(f"  오디오: {len(y_audio)/SR:.2f}초")
#     except:
#         y_audio = np.zeros(int(duration * SR))
#         print(f"  ⚠️ 오디오 로드 실패, 더미 사용")
    
#     # 4. 문장별 멀티모달 분석
#     print("\n[4/6] 🔍 문장별 분석 중...")
    
#     analysis_results = []
    
#     for idx, stt_seg in enumerate(stt_segments):
#         # STT 형식에 따라 키 이름 조정
#         start_sec = stt_seg.get("start", stt_seg.get("start_sec", 0))
#         end_sec = stt_seg.get("end", stt_seg.get("end_sec", 0))
#         text = stt_seg.get("text", stt_seg.get("corrected", ""))
        
#         if end_sec <= start_sec:
#             continue
        
#         # ----- Gesture 분석 -----
#         start_frame = int(start_sec * fps)
#         end_frame = int(end_sec * fps)
        
#         gesture_probs = []
#         for f_start in range(start_frame, max(start_frame + 1, end_frame - CLIP_LEN), STRIDE):
#             frames = []
#             for i in range(f_start, min(f_start + CLIP_LEN, end_frame)):
#                 cap.set(cv2.CAP_PROP_POS_FRAMES, i)
#                 ret, frame = cap.read()
#                 if ret:
#                     frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#                     frame = cv2.resize(frame, RESIZE_HW)
#                     frames.append(frame)
            
#             if len(frames) >= CLIP_LEN // 2:
#                 # 패딩
#                 while len(frames) < CLIP_LEN:
#                     frames.append(frames[-1])
                
#                 vid_np = np.array(frames[:CLIP_LEN], dtype=np.float32) / 255.0
#                 vid_np = np.transpose(vid_np, (3, 0, 1, 2))
#                 vid_np = (vid_np - 0.5) / 0.5
#                 vid_tensor = torch.from_numpy(vid_np).unsqueeze(0).to(device)
                
#                 with torch.no_grad():
#                     g_logits = gesture_model(vid_tensor)
#                     g_prob = F.softmax(g_logits, dim=1)[0, 1].item()
#                     gesture_probs.append(g_prob)
        
#         gesture_score = np.mean(gesture_probs) if gesture_probs else 0.0
        
#         # ----- Audio 분석 -----
#         audio_start = int(start_sec * SR)
#         audio_end = int(end_sec * SR)
#         y_clip = y_audio[audio_start:audio_end]
        
#         if len(y_clip) > 0:
#             mel_feat = extract_mel_spectrogram(y_clip, SR, N_MELS, N_FFT, HOP_LENGTH)
#             mel_feat = pad_or_truncate(mel_feat, WINDOW_SIZE)
#             mel_feat = (mel_feat - mel_feat.mean()) / (mel_feat.std() + 1e-6)
            
#             aud_tensor = torch.from_numpy(mel_feat).float().unsqueeze(0).to(device)
            
#             with torch.no_grad():
#                 a_logits = audio_model(aud_tensor)
#                 a_probs = F.softmax(a_logits, dim=1)[0]
#                 audio_score = 1.0 - a_probs[0].item()  # 비강조 제외
#         else:
#             audio_score = 0.0
        
#         # ----- Text 분석 -----
#         text_score = 0.0
#         if idx < len(text_tensors):
#             t_tensor = text_tensors[idx]
#             if isinstance(t_tensor, torch.Tensor):
#                 # 텐서 norm 기반 점수
#                 if t_tensor.dim() >= 2:
#                     t_feat = t_tensor.mean(dim=-2) if t_tensor.dim() == 3 else t_tensor
#                 else:
#                     t_feat = t_tensor
#                 text_score = min(1.0, t_feat.norm().item() / 50.0)
        
#         # ----- Fusion -----
#         fusion_score = (
#             WEIGHTS["gesture"] * gesture_score +
#             WEIGHTS["audio"] * audio_score +
#             WEIGHTS["text"] * text_score
#         )
        
#         analysis_results.append({
#             "index": idx + 1,
#             "start_sec": round(start_sec, 2),
#             "end_sec": round(end_sec, 2),
#             "text": text,
#             "gesture_score": round(gesture_score, 4),
#             "audio_score": round(audio_score, 4),
#             "text_score": round(text_score, 4),
#             "emphasis_score": round(fusion_score, 4),
#             "is_emphasis": bool(fusion_score >= EMPHASIS_THRESHOLD)
#         })
#         start_frame += STRIDE
#         # 진행률
#         if (idx + 1) % 5 == 0:
#             print(f"  진행: {idx + 1}/{len(stt_segments)}")
    
#     cap.release()
#     print(f"  ✅ {len(analysis_results)}개 문장 분석 완료")
    
#     # 5. 강조 구간 추출 + 피드백 생성
#     print("\n[5/6] 🤖 피드백 생성...")
    
#     emphasis_segments = [r for r in analysis_results if r["is_emphasis"]]
#     print(f"  강조 구간: {len(emphasis_segments)}개")
    
#     # LLM 피드백 시도
#     llm_feedback = None
#     if USE_OPENAI and OPENAI_API_KEY != "your-api-key-here":
#         print("  OpenAI API로 피드백 생성 중...")
#         llm_feedback = generate_llm_feedback_openai(emphasis_segments, duration)
#     elif not USE_OPENAI and ANTHROPIC_API_KEY != "your-api-key-here":
#         print("  Anthropic API로 피드백 생성 중...")
#         llm_feedback = generate_llm_feedback_anthropic(emphasis_segments, duration)
    
#     if llm_feedback is None:
#         print("  ⚠️ LLM API 사용 불가, 로컬 피드백 생성...")
#         llm_feedback = generate_feedback_local(emphasis_segments)
    
#     # 피드백을 분석 결과에 병합
#     final_emphasis_data = []
#     for i, seg in enumerate(emphasis_segments):
#         result = {
#             "index": i + 1,
#             "start_sec": seg["start_sec"],
#             "end_sec": seg["end_sec"],
#             "text": seg["text"],
#             "emphasis_score": seg["emphasis_score"],
#             "key_word": find_key_word(seg["text"]),
#             "reason": ""
#         }
        
#         # LLM 피드백이 있으면 병합
#         if isinstance(llm_feedback, list) and i < len(llm_feedback):
#             fb = llm_feedback[i]
#             if isinstance(fb, dict):
#                 result["key_word"] = fb.get("key_word", result["key_word"])
#                 result["reason"] = fb.get("reason", "")
#                 if "advice" in fb:
#                     result["advice"] = fb["advice"]
        
#         final_emphasis_data.append(result)
    
#     print(f"  ✅ 피드백 생성 완료")
    
#     # 6. 시각화 영상 생성
#     print("\n[6/6] 🎥 시각화 영상 생성...")
#     create_visualization_video(VIDEO_PATH, final_emphasis_data, OUTPUT_VIDEO)
    
#     # 결과 저장
#     print("\n📁 결과 저장...")
    
#     # 전체 분석 결과
#     full_result = {
#         "video_path": VIDEO_PATH,
#         "duration_sec": round(duration, 2),
#         "analysis_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
#         "weights": WEIGHTS,
#         "threshold": EMPHASIS_THRESHOLD,
#         "total_sentences": len(analysis_results),
#         "emphasis_count": len(emphasis_segments),
#         "statistics": {
#             "gesture_mean": round(np.mean([r["gesture_score"] for r in analysis_results]), 4),
#             "audio_mean": round(np.mean([r["audio_score"] for r in analysis_results]), 4),
#             "text_mean": round(np.mean([r["text_score"] for r in analysis_results]), 4),
#             "emphasis_mean": round(np.mean([r["emphasis_score"] for r in analysis_results]), 4),
#         },
#         "total_score": round(np.mean([r["emphasis_score"] for r in analysis_results]) * 100, 2),
#         "emphasis_segments": final_emphasis_data,
#         "all_sentences": analysis_results
#     }
    
#     with open(OUTPUT_JSON, 'w', encoding='utf-8') as f:
#         json.dump(full_result, f, indent=2, ensure_ascii=False)
#     print(f"  ✅ 분석 결과: {OUTPUT_JSON}")
    
#     # LLM 피드백 별도 저장
#     with open(OUTPUT_FEEDBACK, 'w', encoding='utf-8') as f:
#         json.dump(final_emphasis_data, f, indent=2, ensure_ascii=False)
#     print(f"  ✅ 피드백: {OUTPUT_FEEDBACK}")
    
#     # 최종 요약
#     print("\n" + "="*70)
#     print("📊 분석 완료!")
#     print("="*70)
#     print(f"  영상 길이: {duration:.2f}초")
#     print(f"  총 문장: {len(analysis_results)}개")
#     print(f"  강조 구간: {len(emphasis_segments)}개 ({len(emphasis_segments)/len(analysis_results)*100:.1f}%)")
#     print(f"  총점: {full_result['total_score']:.2f} / 100")
#     print(f"\n📋 강조 구간 (처음 5개):")
    
#     for seg in final_emphasis_data[:5]:
#         print(f"  [{seg['index']}] {seg['start_sec']:.1f}s~{seg['end_sec']:.1f}s "
#               f"(점수: {seg['emphasis_score']:.2f})")
#         print(f"      텍스트: {seg['text'][:40]}...")
#         print(f"      키워드: {seg['key_word']}, 이유: {seg['reason'][:30]}...")
    
#     print(f"\n📁 출력 파일:")
#     print(f"  - 분석 결과: {OUTPUT_JSON}")
#     print(f"  - 피드백: {OUTPUT_FEEDBACK}")
#     print(f"  - 시각화 영상: {OUTPUT_VIDEO}")
    
#     print("\n🎉 완료!")
    
#     return full_result


# if __name__ == "__main__":
#     main()

🖥️ Device: cuda

🎬 멀티모달 발표 분석 시스템

[1/6] 📦 모델 로드...
  ✅ Gesture Model 로드
  ✅ Audio Model 로드
  ✅ Text Tensors 로드: 46개

[2/6] 📝 STT 데이터 로드...
⚠️ STT 파일 없음: /home/stu/ai_project/결과/text.json
  ✅ 0개 문장 로드

[3/6] 🎥 비디오 분석...
  영상: test_jo.mp4
  길이: 142.41초, FPS: 29.97


/tmp/ipykernel_2537931/1807707730.py:510: UserWarning: PySoundFile failed. Trying audioread instead.
  y_audio, _ = librosa.load(VIDEO_PATH, sr=SR)
/home/stu/anaconda3/envs/ai_project_env/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


  오디오: 142.52초

[4/6] 🔍 문장별 분석 중...
  ✅ 0개 문장 분석 완료

[5/6] 🤖 피드백 생성...
  강조 구간: 0개
  ⚠️ LLM API 사용 불가, 로컬 피드백 생성...
  ✅ 피드백 생성 완료

[6/6] 🎥 시각화 영상 생성...
  🎥 영상 생성 중... (4268 frames)
    진행률: 11.7%
    진행률: 23.4%
    진행률: 35.1%


KeyboardInterrupt: 

In [4]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import numpy as np
import warnings
from torch.utils.data import DataLoader, Dataset
from config import (
    GESTURE_CONFIG, AUDIO_CONFIG, FUSION_CONFIG,
    GESTURE_MODEL_PATH, AUDIO_MODEL_PATH, MODEL_DIR, 
    ensure_dirs, AUDIO_SCALER_PATH
)
from utils import (
    prepare_all_entries, train_val_split,
    VideoEmphasisDataset, AudioEmphasisDataset,
    load_gesture_model, load_audio_model, AudioFeatureExtractor
)

# 🔇 경고 메시지 끄기 (Librosa 등)
warnings.filterwarnings("ignore")

# =============================================================================
# 🛠️ 1. 설정 및 데이터 로드
# =============================================================================
CSV_PATH = "/home/stu/ai_project/영상csv/최종csv.csv"
batch_size = 128
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️ Device: {device}")

# ★ [수정됨] 3차원 텐서 처리 기능 추가
class TextEmphasisDataset(Dataset):
    def __init__(self, entries, text_tensor_dir=MODEL_DIR): 
        self.entries = entries
        self.input_dim = 1024  # RoBERTa Large용
        self.global_tensor_path = os.path.join(text_tensor_dir, "text_jo_torch_text.pt")
        self.global_tensors = None
        
        if os.path.exists(self.global_tensor_path):
            try: 
                self.global_tensors = torch.load(self.global_tensor_path, map_location='cpu')
                print(f"📦 텍스트 텐서 로드 성공 ({len(self.global_tensors)}개)")
            except: 
                print("⚠️ 텍스트 텐서 로드 실패")

    def __len__(self): return len(self.entries)
    
    def __getitem__(self, idx):
        x = torch.zeros(self.input_dim) # 기본값
        
        if self.global_tensors is not None and idx < len(self.global_tensors):
            tensor_data = self.global_tensors[idx]
            
            if isinstance(tensor_data, torch.Tensor):
                # ★ [핵심 수정] 차원 축소 로직 강화
                # 3차원인 경우: [1, 128, 1024] -> [128, 1024]
                if tensor_data.dim() == 3:
                    tensor_data = tensor_data.squeeze(0)
                
                # 2차원인 경우: [128, 1024] -> [1024] (평균)
                if tensor_data.dim() == 2:
                    x = tensor_data.mean(dim=0)
                elif tensor_data.dim() == 1:
                    x = tensor_data
                    
        y = torch.tensor(self.entries[idx]['label'], dtype=torch.float)
        return x, y

class FusionDataset(Dataset):
    def __init__(self, entries, audio_extractor):
        self.entries = entries
        self.video_dataset = VideoEmphasisDataset(entries, clip_len=16, resize_hw=(112, 112))
        self.audio_dataset = AudioEmphasisDataset(entries, extractor=audio_extractor, config=AUDIO_CONFIG)
        self.text_dataset = TextEmphasisDataset(entries)

    def __len__(self): return len(self.entries)
    def __getitem__(self, idx):
        X_vid, y_vid = self.video_dataset[idx]
        X_aud, _ = self.audio_dataset[idx]
        X_txt, _ = self.text_dataset[idx]
        return X_vid, X_aud, X_txt, y_vid.float()

# 데이터 준비
audio_extractor = AudioFeatureExtractor(AUDIO_CONFIG)
if os.path.exists(AUDIO_SCALER_PATH): audio_extractor.load_scaler(AUDIO_SCALER_PATH)

all_entries = prepare_all_entries(CSV_PATH)
train_entries, val_entries = train_val_split(all_entries, split_ratio=0.8)

# 클래스 가중치
labels = np.array([e['label'] for e in all_entries])
pos_weight_val = np.sum(labels==0) / np.sum(labels==1) if np.sum(labels==1) > 0 else 1.0
pos_weight = torch.tensor([pos_weight_val]).to(device)

train_loader = DataLoader(FusionDataset(train_entries, audio_extractor), batch_size=batch_size, shuffle=True, num_workers=8)
val_loader = DataLoader(FusionDataset(val_entries, audio_extractor), batch_size=batch_size, shuffle=False, num_workers=8)
print(f"✅ 데이터 준비 완료: Train {len(train_loader)} batches")

# =============================================================================
# 🤖 2. 전문가 모델 로드
# =============================================================================
class TextBERTModel(nn.Module):
    def __init__(self, input_dim=1024): 
        super().__init__()
        self.fc = nn.Linear(input_dim, 2)
    def forward(self, x, return_feature=False):
        return (self.fc(x), x) if return_feature else self.fc(x)

g_model = load_gesture_model(GESTURE_MODEL_PATH, device)
a_model = load_audio_model(AUDIO_MODEL_PATH, device)
t_model = TextBERTModel(input_dim=1024).to(device)

if torch.cuda.device_count() > 1:
    g_model = nn.DataParallel(g_model); a_model = nn.DataParallel(a_model); t_model = nn.DataParallel(t_model)

for m in [g_model, a_model, t_model]:
    for p in m.parameters(): p.requires_grad = False
    m.eval()
print("✅ 전문가 모델 로드 및 Freeze 완료")

# =============================================================================
# 🧠 3. Fusion 모델 정의 (3가지)
# =============================================================================
# ① Simple MLP
class SimpleFusionMLP(nn.Module):
    def __init__(self, g_dim=512, a_dim=128, t_dim=1024, hidden_dim=256):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(g_dim+a_dim+t_dim, hidden_dim), nn.BatchNorm1d(hidden_dim), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(hidden_dim, hidden_dim//2), nn.ReLU(), nn.Linear(hidden_dim//2, 1)
        )
    def forward(self, g, a, t): return self.net(torch.cat([g, a, t], dim=1))

# ② Gated Fusion
class GatedFusionNet(nn.Module):
    def __init__(self, g_dim=512, a_dim=128, t_dim=1024, hidden_dim=256):
        super().__init__()
        self.g_proj = nn.Linear(g_dim, hidden_dim); self.a_proj = nn.Linear(a_dim, hidden_dim); self.t_proj = nn.Linear(t_dim, hidden_dim)
        self.gate_fc = nn.Linear(hidden_dim*3, 3)
        self.classifier = nn.Sequential(nn.Linear(hidden_dim, hidden_dim//2), nn.ReLU(), nn.Dropout(0.2), nn.Linear(hidden_dim//2, 1))
    def forward(self, g, a, t):
        g_h, a_h, t_h = F.relu(self.g_proj(g)), F.relu(self.a_proj(a)), F.relu(self.t_proj(t))
        gates = F.softmax(self.gate_fc(torch.cat([g_h, a_h, t_h], dim=1)), dim=1)
        return self.classifier(gates[:,0:1]*g_h + gates[:,1:2]*a_h + gates[:,2:3]*t_h)

# ③ Transformer Fusion
class TransformerFusionNet(nn.Module):
    def __init__(self, g_dim=512, a_dim=128, t_dim=1024, embed_dim=256):
        super().__init__()
        self.projs = nn.ModuleList([nn.Linear(d, embed_dim) for d in [g_dim, a_dim, t_dim]])
        self.cls_token = nn.Parameter(torch.randn(1, 1, embed_dim))
        self.transformer = nn.TransformerEncoder(nn.TransformerEncoderLayer(embed_dim, 4, batch_first=True), 2)
        self.classifier = nn.Linear(embed_dim, 1)
    def forward(self, g, a, t):
        emb = torch.stack([l(x) for l, x in zip(self.projs, [g, a, t])], dim=1)
        seq = torch.cat([self.cls_token.expand(g.size(0), -1, -1), emb], dim=1)
        return self.classifier(self.transformer(seq)[:, 0, :])

# =============================================================================
# 🚀 4. 학습 및 비교 실행
# =============================================================================
def train_eval(name, model, epochs=5):
    print(f"\n⚡ [{name}] Training...")
    if torch.cuda.device_count() > 1: model = nn.DataParallel(model)
    model = model.to(device)
    opt = optim.Adam(model.parameters(), lr=1e-4)
    crit = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    
    best_acc = 0.0
    for ep in range(epochs):
        model.train()
        for v, a, t, y in train_loader:
            v, a, t, y = v.to(device), a.to(device), t.to(device), y.to(device).unsqueeze(1)
            with torch.no_grad(): _, gf = g_model(v, True); _, af = a_model(a, True); _, tf = t_model(t, True)
            opt.zero_grad(); loss = crit(model(gf, af, tf), y); loss.backward(); opt.step()
            
        model.eval()
        corr = 0; total = 0
        with torch.no_grad():
            for v, a, t, y in val_loader:
                v, a, t, y = v.to(device), a.to(device), t.to(device), y.to(device).unsqueeze(1)
                _, gf = g_model(v, True); _, af = a_model(a, True); _, tf = t_model(t, True)
                preds = (torch.sigmoid(model(gf, af, tf)) >= 0.5).float()
                corr += (preds == y).sum().item(); total += y.size(0)
        acc = 100*corr/total
        if acc > best_acc: best_acc = acc
        print(f"   Ep {ep+1}: Val Acc {acc:.2f}%")
    return {"Model": name, "Best Acc": best_acc}

results = []
for name, model in [
    ("Simple MLP", SimpleFusionMLP()), 
    ("Gated Fusion", GatedFusionNet()), 
    ("Transformer", TransformerFusionNet())
]:
    results.append(train_eval(name, model))

print("\n" + "="*40 + "\n🏆 최종 결과 비교\n" + "="*40)
print(pd.DataFrame(results))

🖥️ Device: cuda
[INFO] Loaded 2930 entries from /home/stu/ai_project/영상csv/최종csv.csv
[INFO] Generated 2845 negative entries
[INFO] Total entries: 5775 (pos: 2930, neg: 2845)
📦 텍스트 텐서 로드 성공 (46개)
📦 텍스트 텐서 로드 성공 (46개)
✅ 데이터 준비 완료: Train 37 batches
Loaded gesture model from: /home/stu/ai_project/모델집합/gesture_model.pt
[INFO] Loaded audio model from: /home/stu/ai_project/모델집합/voice_model.pth
✅ 전문가 모델 로드 및 Freeze 완료

⚡ [Simple MLP] Training...


[h264 @ 0x29a47540] mmco: unref short failure
[h264 @ 0x2947a8c0] mmco: unref short failure
[h264 @ 0x2947a8c0] mmco: unref short failure
[h264 @ 0x2947a8c0] mmco: unref short failure
[h264 @ 0x2947a8c0] mmco: unref short failure
[h264 @ 0x2947a8c0] mmco: unref short failure
[h264 @ 0x2947a8c0] mmco: unref short failure
[h264 @ 0x2947a8c0] mmco: unref short failure
[h264 @ 0x2947a8c0] mmco: unref short failure
[h264 @ 0x2947a8c0] mmco: unref short failure
[h264 @ 0x2947a8c0] mmco: unref short failure
[h264 @ 0x2947a8c0] mmco: unref short failure
[h264 @ 0x2947a8c0] mmco: unref short failure
[h264 @ 0x2947a8c0] mmco: unref short failure
[h264 @ 0x2947a8c0] mmco: unref short failure
[h264 @ 0x2947a8c0] mmco: unref short failure
[h264 @ 0x2947a8c0] mmco: unref short failure
[h264 @ 0x2947a8c0] mmco: unref short failure
[h264 @ 0x2947a8c0] mmco: unref short failure
[h264 @ 0x2947a8c0] mmco: unref short failure
[h264 @ 0x2947a8c0] mmco: unref short failure
[h264 @ 0x2947a8c0] mmco: unref sh

   Ep 1: Val Acc 84.94%


[h264 @ 0x35e33c40] mmco: unref short failure
[h264 @ 0x35e33c40] mmco: unref short failure
[h264 @ 0x35e33c40] mmco: unref short failure
[h264 @ 0x35e33c40] mmco: unref short failure
[h264 @ 0x35e33c40] mmco: unref short failure
[h264 @ 0x35e33c40] mmco: unref short failure
[h264 @ 0x35e33c40] mmco: unref short failure
[h264 @ 0x35e33c40] mmco: unref short failure
[h264 @ 0x35e33c40] mmco: unref short failure
[h264 @ 0x35e33c40] mmco: unref short failure
[h264 @ 0x35e33c40] mmco: unref short failure
[h264 @ 0x35e33c40] mmco: unref short failure
[h264 @ 0x35e33c40] mmco: unref short failure
[h264 @ 0x35e33c40] mmco: unref short failure
[h264 @ 0x35e33c40] mmco: unref short failure
[h264 @ 0x36f87c00] mmco: unref short failure
[h264 @ 0x36f87c00] mmco: unref short failure
[h264 @ 0x36f87c00] mmco: unref short failure
[h264 @ 0x36f87c00] mmco: unref short failure
[h264 @ 0x36f87c00] mmco: unref short failure
[h264 @ 0x36f87c00] mmco: unref short failure
[h264 @ 0x36f87c00] mmco: unref sh

   Ep 2: Val Acc 85.97%


...

   Ep 3: Val Acc 87.97%


[h264 @ 0x33f11ac0] mmco: unref short failure
[h264 @ 0x33f11ac0] mmco: unref short failure
[h264 @ 0x33f11ac0] mmco: unref short failure
[h264 @ 0x33f11ac0] mmco: unref short failure
[h264 @ 0x33f11ac0] mmco: unref short failure
[h264 @ 0x33f11ac0] mmco: unref short failure
[h264 @ 0x33f11ac0] mmco: unref short failure
[h264 @ 0x33f11ac0] mmco: unref short failure
[h264 @ 0x33f11ac0] mmco: unref short failure
[h264 @ 0x33f11ac0] mmco: unref short failure
[h264 @ 0x33f11ac0] mmco: unref short failure
[h264 @ 0x33f11ac0] mmco: unref short failure
[h264 @ 0x33f11ac0] mmco: unref short failure
[h264 @ 0x33f11ac0] mmco: unref short failure
[h264 @ 0x33f11ac0] mmco: unref short failure
[h264 @ 0x33f11ac0] mmco: unref short failure
[h264 @ 0x33d2ca00] mmco: unref short failure
[h264 @ 0x33d2ca00] mmco: unref short failure
[h264 @ 0x33d2ca00] mmco: unref short failure
[h264 @ 0x33d2ca00] mmco: unref short failure
[h264 @ 0x33d2ca00] mmco: unref short failure
[h264 @ 0x33d2ca00] mmco: unref sh

   Ep 4: Val Acc 89.35%


[h264 @ 0x352239c0] mmco: unref short failure
[h264 @ 0x33e5e640] mmco: unref short failure
[h264 @ 0x33e5e640] mmco: unref short failure
[h264 @ 0x33e5e640] mmco: unref short failure
[h264 @ 0x33e5e640] mmco: unref short failure
[h264 @ 0x33e5e640] mmco: unref short failure
[h264 @ 0x33e5e640] mmco: unref short failure
[h264 @ 0x33e5e640] mmco: unref short failure
[h264 @ 0x33e5e640] mmco: unref short failure
[h264 @ 0x33e5e640] mmco: unref short failure
[h264 @ 0x33e5e640] mmco: unref short failure
[h264 @ 0x33e5e640] mmco: unref short failure
[h264 @ 0x33e5e640] mmco: unref short failure
[h264 @ 0x33e5e640] mmco: unref short failure
[h264 @ 0x33e5e640] mmco: unref short failure
[h264 @ 0x33e5e640] mmco: unref short failure
[h264 @ 0x33e5e640] mmco: unref short failure
[h264 @ 0x341eec40] mmco: unref short failure
[h264 @ 0x341eec40] mmco: unref short failure
[h264 @ 0x341eec40] mmco: unref short failure
[h264 @ 0x341eec40] mmco: unref short failure
[h264 @ 0x341eec40] mmco: unref sh

   Ep 5: Val Acc 88.74%

🏆 최종 결과 비교
          Model   Best Acc
0    Simple MLP  90.303030
1  Gated Fusion  86.493506
2   Transformer  89.350649


In [ ]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset, DataLoader

# ====================================================
# 0. 사전 설정 (CONFIG & Device)
# ====================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 장치: {device}")

CONFIG = {
    'batch_size': 4,       # 배치 크기 (메모리에 맞게 조절)
    'num_classes': 5,      # 분류할 클래스 개수 (예: 강조, 비강조 등)
    'num_frames': 16,      # 3D CNN 입력 프레임 수
    'image_size': 112      # 입력 이미지 크기
}

# ====================================================
# 1. 모델 클래스 정의 (GestureModel3D)
# ====================================================
class GestureModel3D(nn.Module):
    def __init__(self, backbone_name='r3d_18', num_classes=CONFIG['num_classes']):
        super(GestureModel3D, self).__init__()
        
        # torchvision에서 제공하는 3D CNN 모델 불러오기
        if backbone_name == 'r3d_18':
            self.backbone = torchvision.models.video.r3d_18(pretrained=True)
        elif backbone_name == 'mc3_18':
            self.backbone = torchvision.models.video.mc3_18(pretrained=True)
        elif backbone_name == 'r2plus1d_18':
            self.backbone = torchvision.models.video.r2plus1d_18(pretrained=True)
        else:
            raise ValueError(f"지원하지 않는 모델입니다: {backbone_name}")
        
        # 마지막 FC 레이어를 우리 데이터 클래스 수에 맞게 교체
        in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.backbone(x)

# ====================================================
# 2. (임시) 데이터셋 정의 및 로더 생성
# ====================================================
# 실제 데이터가 있다면 이 부분을 실제 Dataset 클래스로 교체하세요.
class DummyVideoDataset(Dataset):
    def __init__(self, length=200):
        self.length = length
        
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        # 3D CNN 입력 형태: (C, Frames, H, W) -> (3, 16, 112, 112)
        # 랜덤한 텐서 생성 (실제 영상 데이터 대신)
        frames = torch.randn(3, CONFIG['num_frames'], CONFIG['image_size'], CONFIG['image_size'])
        label = torch.randint(0, CONFIG['num_classes'], (1,)).item()
        return frames, label

# 데이터셋 인스턴스 생성
print("데이터셋 생성 중...")
train_dataset = DummyVideoDataset(length=200) # 학습용 200개 (실험 비율 테스트용)
val_dataset = DummyVideoDataset(length=50)    # 검증용 50개

# 검증 로더 생성 (val_loader)
val_loader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'], shuffle=False)

print("설정 완료! 이제 아래 실험 코드를 실행하세요.")

사용 장치: cuda
데이터셋 생성 중...
설정 완료! 이제 아래 실험 코드를 실행하세요.


In [ ]:
import torch
import torch.nn as nn
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import DataLoader, Subset, random_split
from sklearn.metrics import accuracy_score, f1_score

# ====================================================
# [설정] 데이터셋 클래스가 있는 파일 import
# ====================================================
# 만약 데이터셋 코드가 'my_dataset.py'라는 파일에 있다면:
# from my_dataset import VideoEmphasisDataset 
# 
# (만약 방금 주피터 노트북 위 셀에 복사해두셨다면, 별도 import 없이 바로 넘어갑니다!)

# ====================================================
# 1. 설정값 (CONFIG)
# ====================================================
CONFIG = {
    'batch_size': 120,       # GPU 메모리에 맞춰 조절 (4~16 추천)
    'num_classes': 2,      # (강조 O / 강조 X) -> 2개 클래스인 경우
    'clip_len': 16,        # 프레임 수 (Dataset 기본값과 일치)
    'image_size': 112,     # 이미지 크기
    'learning_rate': 1e-4
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 사용 장치: {device}")

# ====================================================
# 2. 모델 정의 (GestureModel3D)
# ====================================================
class GestureModel3D(nn.Module):
    def __init__(self, backbone_name='r3d_18', num_classes=CONFIG['num_classes']):
        super(GestureModel3D, self).__init__()
        
        if backbone_name == 'r3d_18':
            self.backbone = torchvision.models.video.r3d_18(weights='DEFAULT')
        elif backbone_name == 'mc3_18':
            self.backbone = torchvision.models.video.mc3_18(weights='DEFAULT')
        else:
            raise ValueError("지원하지 않는 모델입니다.")
        
        # FC 레이어 교체
        in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.backbone(x)

print("✅ 모델 및 설정 준비 완료.")

🚀 사용 장치: cuda
✅ 모델 및 설정 준비 완료.


In [ ]:
import os
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset

# =============================================================================
# 1. 이 코드를 먼저 실행하세요 (Dataset 클래스 정의)
# =============================================================================
class VideoEmphasisDataset(Dataset):
    def __init__(self, entries, clip_len=16, resize_hw=(112, 112)):
        self.entries = entries
        self.clip_len = clip_len
        self.resize_hw = resize_hw

    def __len__(self):
        return len(self.entries)

    def __getitem__(self, idx):
        entry = self.entries[idx]
        video_path = entry["video_path"]
        start_sec = entry["start_sec"]
        end_sec = entry["end_sec"]
        label = entry["label"]

        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            # 영상이 안 열리면 0으로 채운 텐서 반환 (에러 방지)
            frames = np.zeros((3, self.clip_len, self.resize_hw[0], self.resize_hw[1]), dtype=np.float32)
            return torch.from_numpy(frames), torch.tensor(label, dtype=torch.long)

        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        start_frame = int(start_sec * fps)
        end_frame = int(end_sec * fps)

        start_frame = max(0, min(start_frame, total_frames - 1))
        end_frame = max(0, min(end_frame, total_frames - 1))

        if end_frame <= start_frame:
            end_frame = min(start_frame + self.clip_len, total_frames - 1)

        seg_len = end_frame - start_frame + 1

        if seg_len >= self.clip_len:
            indices = np.linspace(start_frame, end_frame, num=self.clip_len, dtype=int)
        else:
            base_indices = np.linspace(start_frame, end_frame, num=seg_len, dtype=int)
            if seg_len > 0:
                repeat = int(np.ceil(self.clip_len / seg_len))
                indices = np.tile(base_indices, repeat)[:self.clip_len]
            else:
                indices = np.zeros(self.clip_len, dtype=int)

        frames = []
        for f_idx in indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, f_idx)
            ret, frame = cap.read()
            if not ret:
                if len(frames) > 0:
                    frame = frames[-1]
                else:
                    frame = np.zeros((self.resize_hw[0], self.resize_hw[1], 3), dtype=np.uint8)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, self.resize_hw)
            frames.append(frame)

        cap.release()

        # 텐서 변환
        frames = np.stack(frames, axis=0)            # (T, H, W, C)
        frames = frames.astype(np.float32) / 255.0   # [0, 1]
        frames = np.transpose(frames, (3, 0, 1, 2))  # (C, T, H, W)
        frames = (frames - 0.5) / 0.5                # [-1, 1]

        x = torch.from_numpy(frames)
        y = torch.tensor(label, dtype=torch.long)
        return x, y

print("✅ VideoEmphasisDataset 클래스가 정의되었습니다. 이제 다음 코드를 실행하세요!")

✅ VideoEmphasisDataset 클래스가 정의되었습니다. 이제 다음 코드를 실행하세요!


In [ ]:
import pandas as pd
import os
from torch.utils.data import DataLoader, Subset, random_split

# ====================================================
# 3. 데이터 준비 (CSV -> Entries 리스트 변환)
# ====================================================

# 1. 경로 설정
csv_path = "/home/stu/ai_project/영상csv/최종csv.csv"

# 2. CSV 파일 읽기
if os.path.exists(csv_path):
    print(f"✅ 파일을 찾았습니다: {csv_path}")
    df = pd.read_csv(csv_path)
    print(f"   -> 데이터프레임 로드 성공! (상위 컬럼: {df.columns.tolist()})")
    
    entries = []
    
    # -------------------------------------------------------------------------
    # 🔥 [중요] 여기 아래 부분을 CSV 컬럼명에 맞게 수정해야 합니다!
    # -------------------------------------------------------------------------
    try:
        for idx, row in df.iterrows():
            # 만약 에러가 나면, 아래 '따옴표 안의 이름'을 CSV 실제 컬럼명과 똑같이 바꾸세요.
            entries.append({
                "video_path": row['video_path'],  # 영상 경로 컬럼명
                "start_sec": row['start_sec'],        # 시작 시간 컬럼명
                "end_sec": row['end_sec'],            # 종료 시간 컬럼명
                "label": row['label']             # 라벨 컬럼명
            })
        print(f"🎉 리스트 변환 완료! 총 {len(entries)}개의 데이터가 준비되었습니다.")
        
    except KeyError as e:
        print(f"\n❌ [오류 발생] CSV에 '{e}' 라는 이름의 컬럼이 없습니다!")
        print(f"👉 현재 CSV의 컬럼 이름들: {df.columns.tolist()}")
        print("   코드의 row['...'] 부분 이름을 위 목록에 있는 것으로 고쳐주세요.")
        entries = [] # 에러나면 초기화

else:
    print(f"❌ 경로에 파일이 없습니다: {csv_path}")
    entries = []

# 3. 데이터셋 및 로더 생성 (데이터가 있을 때만)
if entries:
    # 데이터셋 생성
    full_dataset = VideoEmphasisDataset(entries, clip_len=CONFIG['clip_len'], resize_hw=(112, 112))

    # Train/Val 분리 (8:2)
    train_size = int(0.8 * len(full_dataset))
    val_size = len(full_dataset) - train_size
    
    # 데이터가 너무 적으면 에러나므로 예외처리
    if val_size == 0:
        print("⚠️ 데이터가 너무 적어서 나누지 않고 전체를 학습에 사용합니다.")
        train_dataset = full_dataset
        val_dataset = full_dataset
    else:
        train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

    # 검증용 로더 (고정)
    val_loader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'], shuffle=False, num_workers=2)

    print("✅ 데이터셋 및 로더 준비 완료! 이제 실험 코드를 실행하세요.")
else:
    print("⛔ 데이터가 로드되지 않아 다음 단계로 진행할 수 없습니다.")

✅ 파일을 찾았습니다: /home/stu/ai_project/영상csv/최종csv.csv
   -> 데이터프레임 로드 성공! (상위 컬럼: ['video_id', 'video_path', 'start_sec', 'end_sec', 'label'])
🎉 리스트 변환 완료! 총 2930개의 데이터가 준비되었습니다.
✅ 데이터셋 및 로더 준비 완료! 이제 실험 코드를 실행하세요.


In [ ]:
# ====================================================
# 4. 실험 시작
# ====================================================
RATIOS = [0.2, 0.4, 0.6, 0.8, 1.0]
EPOCHS_PER_RATIO = 3
MODELS_TO_COMPARE = ["r3d_18", "mc3_18"]
experiment_results = []

print(f"📊 실험 시작: 데이터 비율 {RATIOS}")

for ratio in RATIOS:
    # 데이터셋 서브셋 생성 (비율만큼 랜덤 샘플링)
    total_len = len(train_dataset)
    if total_len == 0: break # 데이터 없으면 중단
    
    subset_len = int(total_len * ratio)
    if subset_len == 0: subset_len = 1 # 최소 1개는 보장
    
    indices = np.random.choice(total_len, subset_len, replace=False)
    subset = Subset(train_dataset, indices)
    
    curr_loader = DataLoader(subset, batch_size=CONFIG['batch_size'], shuffle=True, num_workers=2)
    print(f"\n⚡ [Data: {int(ratio*100)}%] 샘플 수: {len(subset)}개")

    for model_name in MODELS_TO_COMPARE:
        # 모델 초기화
        model = GestureModel3D(backbone_name=model_name).to(device)
        optimizer = torch.optim.AdamW(model.parameters(), lr=CONFIG['learning_rate'])
        criterion = nn.CrossEntropyLoss()
        
        # 학습
        model.train()
        for ep in range(EPOCHS_PER_RATIO):
            for x, y in curr_loader:
                x, y = x.to(device), y.to(device)
                optimizer.zero_grad()
                out = model(x)
                loss = criterion(out, y)
                loss.backward()
                optimizer.step()
        
        # 평가
        model.eval()
        preds_list, targets_list = [], []
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                out = model(x)
                preds = torch.argmax(out, dim=1).cpu().numpy()
                targets = y.cpu().numpy()
                preds_list.extend(preds)
                targets_list.extend(targets)
        
        acc = accuracy_score(targets_list, preds_list)
        f1 = f1_score(targets_list, preds_list, average='weighted', zero_division=0)
        
        experiment_results.append({
            "Ratio": int(ratio * 100),
            "Model": model_name,
            "Accuracy": acc,
            "F1-Score": f1
        })
        print(f"   👉 {model_name}: Acc {acc:.4f}, F1 {f1:.4f}")

# ====================================================
# 5. 결과 시각화
# ====================================================
if experiment_results:
    df = pd.DataFrame(experiment_results)
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    sns.lineplot(ax=axes[0], data=df, x="Ratio", y="Accuracy", hue="Model", marker='o')
    axes[0].set_title("Accuracy by Data Size")
    axes[0].set_ylim(0, 1.0)
    
    sns.lineplot(ax=axes[1], data=df, x="Ratio", y="F1-Score", hue="Model", marker='o')
    axes[1].set_title("F1-Score by Data Size")
    axes[1].set_ylim(0, 1.0)
    
    plt.show()
    print(df)
else:
    print("⚠️ 결과가 없습니다. 데이터를 로드했는지 확인해주세요.")

📊 실험 시작: 데이터 비율 [0.2, 0.4, 0.6, 0.8, 1.0]

⚡ [Data: 20%] 샘플 수: 468개


[h264 @ 0x11dd2100] mmco: unref short failure
[h264 @ 0x11dd2100] mmco: unref short failure
[h264 @ 0x11dd2100] mmco: unref short failure
[h264 @ 0x11dd2100] mmco: unref short failure
[h264 @ 0x11c66b40] mmco: unref short failure
[h264 @ 0x11c66b40] mmco: unref short failure


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.44 GiB. GPU 0 has a total capacity of 22.37 GiB of which 423.19 MiB is free. Process 3886228 has 1.45 GiB memory in use. Including non-PyTorch memory, this process has 20.50 GiB memory in use. Of the allocated memory 20.32 GiB is allocated by PyTorch, and 23.84 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

: 